In [ ]:
# 3/1/2022
# markstro

# This notebook demos how PRMS makes the snow calculations

In [ ]:
import pandas as pd
import numpy as np
import gc
import sys
import os
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
import math

In [ ]:
pfn = "C:/Users/markstro/jupyter_notebooks/EC/hru1/input/hru1.params"
outdir = "C:/Users/markstro/jupyter_notebooks/EC/hru1/output/"
indir = "C:/Users/markstro/jupyter_notebooks/EC/hru1/input/"

In [ ]:
# Function that will read PRMS parameter file
# Returns the contents of param file in python data structures
def read_param_file(pfn):
    line_num = 0
    vals = {}
    dims = {}
    param_dims = {}
    param_type = {}

    with open(pfn) as f:
        reading_dims = False
        for line in f:
            try:
                line = line.rstrip()  # remove '\n' at end of line
                line_num += 1
                if line == "** Dimensions **":
                    reading_dims = True
                    line = f.readline().rstrip()
                    line_num += 1

                if line == "** Parameters **":
                    reading_dims = False
                    break

                if reading_dims:
                    line = f.readline().rstrip()
                    line_num += 1
                    dim_name = line

                    line = f.readline().rstrip()
                    line_num += 1
                    size = line

                    if dim_name in dims.keys():
                        pass
                    else:
                        dims[dim_name] = int(size)
            except:
                print("**** read parameters exception line = ", line)
                print("**** read parameters exception line_num = ", str(line_num))
                print("**** Unexpected error:", sys.exc_info()[0])

        #        read params
        for line in f:
            try:
                line = line.rstrip()  # remove '\n' at end of line
                line_num += 1

                if line == "####":
                    line = f.readline().rstrip()
                    line = line.split(" ", 1)[
                        0
                    ]  # old format parameter files have a blank (' ') and then a width format value. Strip this off.
                    param_name = line
                    line_num += 1

                    line = f.readline().rstrip()
                    line_num += 1
                    num_dims = int(line)
                    pd = [None] * num_dims
                    for ii in range(num_dims):
                        line = f.readline().rstrip()
                        pd[ii] = line
                        line_num += 1

                    param_dims[param_name] = pd

                    line = f.readline().rstrip()
                    line_num += 1
                    num_vals = int(line)
                    line = f.readline().rstrip()
                    line_num += 1
                    tp = int(line)
                    param_type[param_name] = tp

                    if tp == 2:
                        vs = np.zeros(num_vals, dtype=np.float)
                        for jj in range(num_vals):
                            line = f.readline().rstrip()
                            line_num += 1
                            vs[jj] = float(line)

                    elif tp == 1:
                        vs = np.zeros(num_vals, dtype=np.int)
                        for jj in range(num_vals):
                            line = f.readline().rstrip()
                            line_num += 1
                            vs[jj] = int(line)

                    else:
                        vs = np.zeros(num_vals, dtype=np.chararray)
                        for jj in range(num_vals):
                            line = f.readline().rstrip()
                            line_num += 1
                            vs[jj] = line

                    if num_dims == 2:
                        vs.shape = (dims[pd[1]], dims[pd[0]])

                    if param_name in vals.keys():
                        print("parameter ", param_name, " is already in ", pfn)
                    else:
                        vals[param_name] = vs

            except:
                print("read parameters exception line_num = ", str(line_num))
                print("Unexpected error:", sys.exc_info()[0])

    gc.collect()

    return (dims, vals, param_dims, param_type)

In [ ]:
# I don't understand why I needed to write this function myself
# Input: datetime.Date
# Output: day of the year (aka "Julian date")
def day_of_year(d):
    N1 = math.floor(275 * d.month / 9)
    N2 = math.floor((d.month + 9) / 12)
    N3 = 1 + math.floor((d.year - 4 * math.floor(d.year / 4) + 2) / 3)
    N = N1 - (N2 * N3) + d.day - 30
    return N

# Read the simulated output into a dataframe

In [ ]:
# This loops over all of the csv files in the output directory

sim_vals = None
for filename in os.listdir(outdir):
    f = os.path.join(outdir, filename)
    if os.path.isfile(f) and f.endswith(".csv"):
        print("reading", f)
        varname = filename.split(".")[0]

        if sim_vals is None:
            sim_vals = pd.read_csv(f, names=["datestr", varname], skiprows=1)

        else:
            if varname != "stats":
                df = pd.read_csv(f, names=["datestr", varname], skiprows=1)
                sim_vals = pd.merge(
                    sim_vals, df[["datestr", varname]], on="datestr", how="left"
                )

sim_vals["date"] = [
    datetime.strptime(date, "%Y-%m-%d").date() for date in sim_vals["datestr"]
]
del sim_vals["datestr"]

In [ ]:
sim_vals.head()

# Read the CBH forcing data into the dataframe

In [ ]:
def add_date_for_PRMS_dates(df):
    df["date"] = [date(y, m, d) for y, m, d in zip(df["yr"], df["mo"], df["da"])]
    del df["yr"]
    del df["mo"]
    del df["da"]
    del df["hr"]
    del df["mi"]
    del df["se"]
    return df

In [ ]:
# Read the CBH forcing inputs
cbh_vals = None
for filename in os.listdir(indir):
    f = os.path.join(indir, filename)
    if os.path.isfile(f) and f.endswith(".cbh"):
        varname = filename.split(".")[0]

        if cbh_vals is None:
            cbh_vals = pd.read_csv(
                f,
                names=["yr", "mo", "da", "hr", "mi", "se", varname],
                skiprows=3,
                delim_whitespace=True,
            )
            cbh_vals = add_date_for_PRMS_dates(cbh_vals)

        else:
            foo = pd.read_csv(
                f,
                names=["yr", "mo", "da", "hr", "mi", "se", varname],
                skiprows=3,
                delim_whitespace=True,
            )
            foo = add_date_for_PRMS_dates(foo)
            cbh_vals = pd.merge(cbh_vals, foo[["date", varname]], on="date", how="left")

# Sync up the dates of the forcings and the outputs. This makes sure that all values are compared for the same date(s)

In [ ]:
synced_vals = pd.merge(cbh_vals, sim_vals, on="date", how="right")

In [ ]:
print(synced_vals.shape)

In [ ]:
synced_vals.head()

In [ ]:
synced_vals.tail()

In [ ]:
synced_vals.info()

# Compute the snow pack

In [ ]:
# Constants
NORTHERN = 1
SOUTHERN = 2
NOT_A_GLACIER_HRU = -1
INCH2CM = 2.54

MAXALB = 15
acum_init = np.array(
    [
        0.80,
        0.77,
        0.75,
        0.72,
        0.70,
        0.69,
        0.68,
        0.67,
        0.66,
        0.65,
        0.64,
        0.63,
        0.62,
        0.61,
        0.60,
    ]
)
amlt_init = np.array(
    [
        0.72,
        0.65,
        0.60,
        0.58,
        0.56,
        0.54,
        0.52,
        0.50,
        0.48,
        0.46,
        0.44,
        0.43,
        0.42,
        0.41,
        0.40,
    ]
)
BARE_SOIL = 0
GRASSES = 1
SHRUBS = 2
TREES = 3
CONIFEROUS = 4
active_glacier = False

In [ ]:
def which_hemisphere(hl, ha):
    basin_lat = sum(hl * ha) / sum(ha)
    if basin_lat > 0.0:
        return NORTHERN
    else:
        return SOUTHERN

In [ ]:
def julian_day_water(hemisphere, doy):
    if hemisphere == NORTHERN:
        jdoy = doy - 274
        if jdoy < 1:
            jdoy = doy + 92
    else:
        print("fix julian_day_water for southern hemisphere")
        jdoy = 0
    return jdoy

In [ ]:
# !***********************************************************************
# !      Subroutine to compute changes in snowpack when a net gain in
# !        heat energy has occurred.
# !***********************************************************************


def calin(
    cal,
    pkwater_equiv,
    pk_def,
    pk_temp,
    pk_ice,
    freeh2o,
    snowcov_area,
    snowmelt,
    pk_depth,
    pss,
    pst,
    iasw,
    pk_den,
    freeh2o_cap,
    den_max,
    ihru_gl,
):
    #     ! Arguments
    #     INTEGER, INTENT(INOUT) :: Iasw
    #     INTEGER, INTENT(IN) :: Ihru_gl
    #     REAL, INTENT(IN) :: Cal, Freeh2o_cap, Snowcov_area, Den_max
    #     REAL, INTENT(INOUT) :: Freeh2o
    #     DOUBLE PRECISION, INTENT(INOUT) :: Pkwater_equiv
    #     REAL, INTENT(INOUT) :: Pk_def, Pk_temp, Pk_ice, Pk_den, Snowmelt
    #     DOUBLE PRECISION, INTENT(INOUT) :: Pss, Pst, Pk_depth

    #   ! Calculate the difference between the incoming calories and the
    #   ! calories needed to bring the pack to isothermal
    #   ! at 0 (heat deficit)
    #   dif = Cal - Pk_def ! [cal/cm^2]
    # ---1---2---3---4---5---6
    dif = cal - pk_def
    print("         calin 1:", dif, cal, pk_def, pk_den)

    #   ! The way incoming heat is handled depends on whether there is
    #   ! not enough, just enough, or more than enough heat to overcome
    #   ! the heat deficit of the snowpack.
    #   ! 3 choices below (if-then, elseif, else)

    #   ! (1) Not enough heat to overcome heat deficit...
    #   IF ( dif<0.0 ) THEN
    # ---1---2---3---4---5---6
    if dif < 0.0:
        #     ! Reduce the heat deficit by the amount of incoming calories
        #     ! and adjust to the new temperature based on new heat deficit
        #     Pk_def = Pk_def - Cal ! [cal/cm^2]
        #     Pk_temp = -Pk_def/SNGL(Pkwater_equiv*1.27D0) ! [degrees C]
        pk_def = pk_def - cal
        pk_temp = -pk_def / pkwater_equiv * 1.27

        print("         calin 2:", pk_def, pkwater_equiv)

    #   ! (3) More than enough heat to overcome heat deficit
    #   !     (melt ice)...
    #   ELSEIF ( dif>0.0 ) THEN
    # ---1---2---3---4---5---6
    elif dif > 0.0:
        #     ! calculate the potential amount of snowmelt from excess
        #     ! heat in rain it takes 203.2 calories / (in cm^2) to melt snow
        #     ! (latent heat of fusion)
        #     ! convert from 1 cm depth over 1 square cm to
        #     ! 1 inch depth over 1 square cm 80.0*(INCH2CM = 2.54 cm/in) = 203.2
        #     pmlt = dif/203.2 ! [inches]
        pmlt = dif / 203.2
        #     ! Actual snowmelt can only come from snow covered area, so to
        #     ! calculate the actual potential snowmelt, the potential
        #     ! snowmelt from snowcovered area must be re-normalized to
        #     ! HRU area (rather than snowcover area)
        #     ! In effect, the potential snowmelt per area is reduced by the
        #     ! fraction of the watershed that is actually covered by snow
        #     apmlt = pmlt*Snowcov_area ! [inches]
        apmlt = pmlt * snowcov_area
        #     ! Set the heat deficit and temperature of the remaining
        #     ! snowpack to 0
        #     Pk_def = 0.0 ! [cal/cm^2]
        #     Pk_temp = 0.0 ! [degrees C]
        pk_def = 0.0
        pk_temp = 0.0
        #     ! The only pack ice that is melted is in the snow covered area,
        #     ! so the pack ice needs to be re-normalized to the snowcovered
        #     ! area (rather than HRU area)
        #     ! In effect, the pack ice per area is increased by the fraction
        #     ! of the watershed that is actually covered by snow
        #     IF ( Snowcov_area>0.0 ) THEN
        #       apk_ice = Pk_ice/Snowcov_area ! [inches]
        #     ELSE
        # !          PRINT *, 'snowcov_area really small, melt all ice', snowcov_area, ' pmlt:', pmlt, ' dif:', dif, ' pk_ice:', Pk_ice
        #       apk_ice = 0.0
        #     ENDIF

        print("         calin 2.1:", pmlt, apmlt, snowcov_area)

        # ---1---2---3---4---5---6
        if snowcov_area > 0.0:
            apk_ice = pk_ice / snowcov_area
        else:
            #     !          PRINT *, 'snowcov_area really small, melt all ice', snowcov_area, ' pmlt:', pmlt, ' dif:', dif, ' pk_ice:', Pk_ice
            apk_ice = 0.0
        #     ! If snow is melting, the heat is handled based on whether all
        #     ! or only part of the pack ice melts
        #     ! 2 options below (if-then, else)

        #     ! (3.1) Heat applied to snow covered area is sufficient
        #     !       to melt all the ice in that snow pack...
        #     ! if on snow over glacier or active_layer and have excess energy from day over
        #     !        depth can melt from layer thickness, add depth to that layer
        # markstro leave glacier code out for now
        #     IF ( Active_glacier>OFF ) THEN
        #       IF ( pmlt>apk_ice ) THEN
        #         !fractionate density with snow/active layer melting vs extra ice underneath melting
        #         Pk_den = Pk_den*SNGL(apk_ice/pmlt) + 0.917*SNGL((pmlt-apk_ice)/pmlt)
        #         apk_ice = pmlt
        #         Pk_ice =  apmlt
        #         Pkwater_equiv = apmlt
        #         Freeh2o = 0.0 ! [inches]
        #         Iasw = 0
        #         Pk_def = 0.0   ! [cal / cm^2]
        #         Pk_temp = 0.0  ! [degreees C]
        #         Pst = 0.0D0      ! [inches]
        #       ENDIF
        #     ENDIF

        #     IF ( pmlt>apk_ice ) TpHEN ! will not happen if Active_glacier>OFF because of above
        # ---1---2---3---4---5---6
        if pmlt > apk_ice:
            #       ! All pack water equivalent becomes meltwater
            #       Snowmelt = Snowmelt + SNGL( Pkwater_equiv ) ! [inches]
            #       Pkwater_equiv = 0.0D0 ! [inches]
            #       Iasw = 0 ! snow area does not change
            snowmelt = snowmelt + pkwater_equiv
            pkwater_equiv = 0.0
            iasw = 0
            #       ! Set all snowpack states to 0
            #       ! Snowcov_area = 0.0 ! [fraction of area] ! shouldn't be changed with melt
            #       Pk_def = 0.0   ! [cal / cm^2]
            #       Pk_temp = 0.0  ! [degreees C]
            #       Pk_ice = 0.0   ! [inches]
            #       Freeh2o = 0.0  ! [inches]
            #       Pk_depth = 0.0D0 ! [inches]
            #       Pss = 0.0D0      ! [inches]
            #       Pst = 0.0D0      ! [inches]
            #       Pk_den = 0.0     ! [fraction of depth]
            pk_def = 0.0
            pk_temp = 0.0
            pk_ice = 0.0
            freeh2o = 0.0
            pk_depth = 0.0
            pss = 0.0
            pst = 0.0
            pk_den = 0.0
            # ---1---2---3---4---5---6
            print("         calin 2.2:", pk_def, pk_ice)

        #     ! (3.2) Heat only melts part of the ice in the snow pack...
        #     ELSE
        # ---1---2---3---4---5---6
        else:
            #       ! Remove actual melt from frozen water and add melt to
            #       ! free water
            #       Pk_ice = Pk_ice - apmlt ! [inches]
            #       Freeh2o = Freeh2o + apmlt ! [inches]
            pk_ice = pk_ice - apmlt
            freeh2o = freeh2o + apmlt
            #       ! Calculate the capacity of the snowpack to hold free water
            #       ! according to its current level of frozen water
            #       pwcap = Freeh2o_cap*Pk_ice ! [inches]
            pwcap = freeh2o_cap * pk_ice
            #       ! Calculate the amount of free water in excess of the
            #       ! capacity to hold free water
            #       dif_dble = DBLE( Freeh2o - pwcap ) ! [inches]
            dif_dble = freeh2o - pwcap
            #       ! If there is more free water than the snowpack can hold,
            #       ! then there is going to be melt...
            #       IF ( dif_dble>0.0D0 ) THEN
            #         IF ( dif_dble>Pkwater_equiv ) dif_dble = Pkwater_equiv
            if dif_dble > 0.0:
                if dif_dble > pkwater_equiv:
                    dif_dble = pkwater_equiv
                #         ! total packwater decreases by the excess and a new depth
                #         ! is calculated based on density
                #         Pkwater_equiv = Pkwater_equiv - dif_dble ! [inches]
                pkwater_equiv = pkwater_equiv - dif_dble
                #         ! free water is at the current capacity
                #         Freeh2o = pwcap ! [inches]
                freeh2o = pwcap
                #         IF ( Pk_den>0.0 ) THEN
                #           Pk_depth = Pkwater_equiv/DBLE(Pk_den) ! [inches
                if pk_den > 0.0:
                    pk_depth = pkwater_equiv / pk_den
                #         !   sure there is no division by zero (this can happen
                #         !   if there is a mixed event on no existing snowpack
                #         !   because a pack density has not been calculated, yet
                #         ELSE
                else:
                    #         !rsr, this should not happen, remove later
                    #           IF ( Print_debug>DEBUG_less ) THEN
                    #             PRINT *, 'snow density problem', Pk_depth, Pk_den, Pss, Pkwater_equiv
                    #             CALL print_date(1)
                    #           ENDIF
                    #           IF ( Active_glacier==OFF ) Pk_den = Den_max
                    #           Pk_depth = Pkwater_equiv/DBLE(Den_max) ! [inches]
                    if active_glacier == False:
                        pk_den = den_max
                        pk_depth = pkwater_equiv / den_max
                    #         ENDIF

                    #         ! snowmelt increases by the excess free water
                    #         Snowmelt = Snowmelt + SNGL( dif_dble ) ! [inches]
                    snowmelt = snowmelt + dif_dble
                    #         ! reset the previous-snowpack-plus-new-snow to the
                    #         ! current pack water equivalent
                    #         Pss = Pkwater_equiv ! [inches]
                    pss = pkwater_equiv
        #       ENDIF
        #     ENDIF

        #   ! (2) Just enough heat to overcome heat deficit
        #   ELSE ! IF ( dif==0.0 ) THEN ! rsr 1/27/2016 why not set all snow states to 0 ???
        print("         calin 3:", pk_def, pkwater_equiv)
    # ---1---2---3---4---5---6
    else:
        #     ! Set temperature and heat deficit to zero
        #     Pk_temp = 0.0 ! [degrees C]
        #     Pk_def = 0.0 ! [cal/cm^2]
        pk_temp = 0.0
        pk_def = 0.0
        print("         calin 4:", pk_def, pkwater_equiv)

    #   ENDIF
    #   IF ( .NOT.(Pkwater_equiv>0.0D0) ) Pk_den = 0.0
    if not pkwater_equiv > 0.0:
        pk_den = 0.0
    #   ! If on melting glacier ice/firn, Ihru_gl >0, so melted active layer (won't melt infinite ice layer)
    #   IF ( Ihru_gl>0) THEN
    #     IF ( .NOT.(Pkwater_equiv>0.0D0) ) CALL glacr_states_to_zero(Ihru_gl,0)
    #   ENDIF

    #   END SUBROUTINE calin

    #     print("         calin 5:", iasw, freeh2o, pkwater_equiv, pk_def, pk_temp, pk_ice, pk_den, snowmelt, pss, pst, pk_depth)
    print("         calin 5:", pk_den)

    return (
        iasw,
        freeh2o,
        pkwater_equiv,
        pk_def,
        pk_temp,
        pk_ice,
        pk_den,
        snowmelt,
        pss,
        pst,
        pk_depth,
    )

In [ ]:
# !***********************************************************************
# !      Subroutine to compute change in snowpack when a net loss in
# !        heat energy has occurred.
# !***********************************************************************


def caloss(cal, pkwater_equiv, pk_def, pk_temp, pk_ice, freeh2o, ihru_gl):
    # ! Arguments
    # INTEGER, INTENT(IN) :: Ihru_gl
    # REAL, INTENT(IN) :: Cal
    # DOUBLE PRECISION, INTENT(INOUT) :: Pkwater_equiv
    # REAL, INTENT(INOUT) :: Pk_def, Pk_ice, Freeh2o, Pk_temp

    #   ! Loss of heat is handled differently if there is liquid water in
    #   ! the snowpack or not
    #   ! 2 options below (if-then, else)

    #   ! (1) No free water exists in pack
    #   IF ( Freeh2o<CLOSEZERO ) THEN
    if freeh2o < 0.0:
        #     ! heat deficit increases because snow is colder than pack
        #     ! (minus a negative number = plus)
        #     Pk_def = Pk_def - Cal ! [cal/cm^2]
        pk_def = pk_def - cal

    #   ! (2) Free water exists in pack
    #   ELSE
    else:
        #     ! calculate the total amount of heat per area that can be
        #     ! released by free water freezing
        #     calnd = Freeh2o*203.2 ! [cal/cm^2]
        calnd = freeh2o * 203.2
        #     ! determine the difference between heat in free water and the
        #     ! heat that can be absorbed by new snow (without melting)
        #     ! remember that cal is a negative number
        #     dif = Cal + calnd ! [cal/cm^2]
        dif = cal + calnd

        #     ! The effect of freezing water depends on whether all or only
        #     ! part of the liquid water freezes
        #     ! 2 options below (if-then, else)

        #     ! (2) Only part of free water freezes
        #     IF ( dif>0.0 ) THEN
        if dif > 0.0:
            #       ! the calories absorbed by the new snow freezes some
            #       ! of the free water
            #       ! (increase in ice, decrease in free water)
            #       Pk_ice = Pk_ice + (-Cal/203.2) ! [inches]
            #       Freeh2o = Freeh2o - (-Cal/203.2) ! [inches]
            #       RETURN
            pk_ice = pk_ice + (-cal / 203.2)
            freeh2o = freeh2o - (-cal / 203.2)
            return pkwater_equiv, pk_def, pk_ice, freeh2o, pk_temp
        #     ! (1) All free water freezes
        #     ELSE ! IF ( dif<=0.0 ) THEN
        else:
            #       ! if all the water freezes, then the remaining heat
            #       ! that can be absorbed by new snow (that which is not
            #       ! provided by freezing free water) becomes the new pack
            #       ! heat deficit
            #       IF ( dif<0.0 ) Pk_def = -dif ! [cal/cm^2]
            if dif < 0.0:
                pk_def = -dif
            #       ! free pack water becomes ice
            #       Pk_ice = Pk_ice + Freeh2o ! [inches]
            #       Freeh2o = 0.0 ! [inches]
            pk_ice = pk_ice + freeh2o
            freeh2o = 0.0

    #     ENDIF
    #   ENDIF

    #   ! if there is still a snowpack, calculate the new temperature
    #   IF ( Pkwater_equiv>0.0D0 ) THEN
    #     Pk_temp = -Pk_def/SNGL(Pkwater_equiv*1.27D0)  ! [degrees C]
    #   ELSE
    if pkwater_equiv > 0.0:
        pk_temp = -pk_def / pkwater_equiv * 1.27

    # !        IF ( Pkwater_equiv<0.0D0 ) THEN
    # !          IF ( Pkwater_equiv<-DNEARZERO ) &
    # !     &         PRINT *, 'snowpack issue 4, negative pkwater_equiv', Pkwater_equiv
    # !          Pkwater_equiv = 0.0D0
    # !        ENDIF
    #     ! If on melting glacier ice/firn, Ihru_gl >0, so melted active layer (won't melt infinite ice layer)
    #     If (Ihru_gl>0) CALL glacr_states_to_zero(Ihru_gl,0)
    #   ENDIF

    #   END SUBROUTINE caloss

    return pkwater_equiv, pk_def, pk_ice, freeh2o, pk_temp

In [ ]:
def ppt_to_pack(
    pptmix,
    iasw,
    tmaxc,
    tminc,
    tavgc,
    pkwater_equiv,
    net_rain,
    pk_def,
    pk_temp,
    pk_ice,
    freeh2o,
    snowcov_area,
    snowmelt,
    pk_depth,
    pss,
    pst,
    net_snow,
    pk_den,
    pptmix_nopack,
    pk_precip,
    tmax_allsnow_c,
    Freeh2o_cap,
    den_max,
    ihru_gl,
):
    # INTEGER, INTENT(IN) :: Pptmix, Ihru_gl
    # INTEGER, INTENT(INOUT) :: Iasw, Pptmix_nopack
    # REAL, INTENT(IN) :: Tmaxc, Tminc, Tavgc, Net_rain, Net_snow
    # REAL, INTENT(IN) :: Freeh2o_cap, Tmax_allsnow_c, Den_max
    # REAL, INTENT(INOUT) :: Snowmelt, Freeh2o, Pk_precip
    # REAL, INTENT(INOUT) :: Pk_def, Pk_ice, Pk_den, Snowcov_area, Pk_temp
    # DOUBLE PRECISION, INTENT(INOUT) :: Pkwater_equiv, Pk_depth, Pst, Pss

    #   ! The temperature of precipitation will be different if it is mixed or
    #   ! all rain or snow 2 options below (if-then, else)

    #   ! If there is any snow, the snow temperature is the average
    #   ! temperature
    #   tsnow = Tavgc ! [degrees C]
    #   ! (1) If precipitation is mixed...
    #   IF ( Pptmix==ACTIVE ) THEN
    #     ! If there is any rain, the rain temperature is halfway between the maximum
    #     ! temperature and the allsnow temperature
    #     train = (Tmaxc+Tmax_allsnow_c)*0.5 ! [degrees C]

    #     ! Temperatures will be different, depending on if there is an
    #     ! existing snowpack or not

    #     ! If there is a snowpack, snow temperature is halfway between
    #     ! the minimum daily temperature and maximum temperature for
    #     ! which all precipitation is snow
    #     IF ( Pkwater_equiv>0.0D0 ) THEN
    #       tsnow = (Tminc+Tmax_allsnow_c)*0.5 ! [degrees C]

    #     ! If there is no existing snowpack, snow temperature is the
    #     ! average temperature for the day
    #     ELSEIF ( Pkwater_equiv<0.0D0 ) THEN
    # !          IF ( Pkwater_equiv<-DNEARZERO ) &
    # !     &         PRINT *, 'snowpack issue in ppt_to_pack, negative pkwater_equiv', Pkwater_equiv
    #       Pkwater_equiv = 0.0D0 ! to be sure negative snowpack is ignored
    #     ENDIF

    #   ! (2) If precipitation is all snow or all rain...
    #   ELSE ! on glacier ice goes in here only
    #     ! If there is any rain, the rain temperature is the average
    #     ! temperature
    #     train = Tavgc ! [degrees C]
    #     ! If average temperature is close to freezing, the rain
    #     ! temperature is halfway between the maximum daily temperature
    #     ! and maximum temperature for which all precipitation is snow
    #     IF ( train<CLOSEZERO ) train = (Tmaxc+Tmax_allsnow_c)*0.5 ! [degrees C]
    #   ENDIF

    # 23456789112345678921234567893123456789
    tsnow = tavgc
    print("      ppt_to_pack 0", tavgc, pptmix, pk_den)
    if pptmix == True:
        train = (tmaxc + tmax_allsnow_c) * 0.5
        if pkwater_equiv > 0.0:
            tsnow = tminc + tmax_allsnow_c * 0.5
        elif pkwater_equiv < 0.0:
            pkwater_equiv = 0.0

        print(
            "      ppt_to_pack 1",
            train,
            tmaxc,
            tmax_allsnow_c,
            pkwater_equiv,
            tsnow,
            tminc,
        )

    else:
        train = tavgc

    #      IF ( train<0.0 ) train = 0.0 ! [degrees C] ! train can't be < 0
    #      IF ( tsnow>0.0 ) tsnow = 0.0 ! [degrees C] ! tsnow can't be > 0
    # 23456789112345678921234567893123456789
    if train < 0.0:
        train = 0.0
    if tsnow > 0.0:
        tsnow = 0.0
    # 23456789112345678921234567893123456789

    #       ! Leavesley comments...
    #       ! If snowpack already exists, add rain first, then add
    #       ! snow.  If no antecedent snowpack, rain is already taken care
    #       ! of, so start snowpack with snow.  This SUBROUTINE assumes
    #       ! that in a mixed event, the rain will be first and turn to
    #       ! snow as the temperature drops.
    #
    #       ! Rain can only add to the snowpack if a previous snowpack
    #       ! exists, so rain or a mixed event is processed differently
    #       ! when a snowpack exists
    #       ! 2 options below (if-then, elseif)
    #
    #       ! (1) If there is net rain on an existing snowpack...
    #       IF ( Pkwater_equiv>0.0D0 ) THEN
    #         IF ( Net_rain>0.0 ) THEN ! on glacier ice goes in here only
    # 23456789112345678921234567893123456789
    print("      ppt_to_pack 2", pkwater_equiv)
    if pkwater_equiv > 0.0:
        print("      ppt_to_pack 3", pkwater_equiv, net_rain)
        if net_rain > 0.0:
            #           ! Add rain water to pack (rain on snow) and increment the
            #           ! precipitation on the snowpack by the rain water
            #           Pkwater_equiv = Pkwater_equiv + DBLE(Net_rain) ! [inches]
            #           Pk_precip = Pk_precip + Net_rain ! [inches]
            pkwater_equiv = pkwater_equiv + net_rain
            pk_precip = pk_precip + net_rain

            #           ! Incoming rain water carries heat that must be added to
            #           ! the snowpack.
            #           ! This heat could both warm the snowpack and melt snow.
            #           ! Handling of this heat depends on the current thermal
            #           ! condition of the snowpack.
            #           ! 2 options below (if-then, else)

            #           ! (1.1) If the snowpack is colder than freezing it has a
            #           ! heat deficit (requires heat to be brought to isothermal
            #           ! at 0 degC)...
            #           IF ( Pk_def>0.0 ) THEN
            if pk_def > 0.0:
                #             ! Calculate the number of calories given up per inch of
                #             ! rain when cooling it from the current rain temperature
                #             ! to 0 deg C and then freezing it (liquid to solid state
                #             ! latent heat)
                #             ! This calculation assumes a volume of an inch of rain
                #             ! over a square cm of area
                #             ! 80 cal come from freezing 1 cm3 at 0 C
                #             ! (latent heat of fusion is 80 cal/cm^3),
                #             ! 1 cal from cooling 1cm3 for every degree C
                #             ! (specific heat of water is 1 cal/(cm^3 degC)),
                #             ! convert from 1 cm depth over 1 square cm to
                #             ! 1 inch depth over 1 square cm (INCH2CM = 2.54 cm/in)
                #             caln = (80.0+train)*INCH2CM ! [cal / (in cm^2)]
                caln = (80.0 + train) * INCH2CM
                #             ! calculate the amount of rain in inches
                #             ! (at the current rain temperature)
                #             ! needed to bring the snowpack to isothermal at 0
                #             pndz = Pk_def/caln ! [inches]
                pndz = pk_def / caln

                #             ! The effect of rain on the snowpack depends on if there
                #             ! is not enough, enough, or more than enough heat in the
                #             ! rain to bring the snowpack to isothermal at 0 degC or not
                #             ! 3 options below (if-then, elseif, else)

                #             ! (1.1.1) Exactly enough rain to bring pack to isothermal...
                #             IF ( ABS(Net_rain-pndz)<CLOSEZERO ) THEN
                # 23456789112345678921234567893123456789
                if net_rain - pndz < 0.0:
                    #               ! Heat deficit and temperature of the snowpack go to 0
                    #               Pk_def = 0.0  ! [cal/cm^2]
                    #               Pk_temp = 0.0 ! [degrees C]
                    pk_def = 0.0
                    pk_temp = 0.0
                    #               ! In the process of giving up its heat, all the net rain
                    #               ! freezes and becomes pack ice
                    #               Pk_ice = Pk_ice + Net_rain ! [inches]
                    pk_ice = pk_ice + net_rain

                #             ! (1.1.2) Rain not sufficient to bring pack to isothermal...
                #             ELSEIF ( Net_rain<pndz ) THEN
                # 23456789112345678921234567893123456789
                elif net_rain < pndz:
                    #               ! The snowpack heat deficit decreases by the heat provided
                    #               ! by rain and a new snowpack temperature is calculated
                    #               ! 1.27 is the specific heat of ice (0.5 cal/(cm^3 degC))
                    #               ! times the conversion of cm to inches (2.54 cm/in)
                    #               Pk_def = Pk_def - (caln*Net_rain) ! [cal/(in cm^3)]
                    #               Pk_temp = -Pk_def/SNGL(Pkwater_equiv*1.27D0)
                    pk_def = pk_def - (caln * net_rain)
                    pk_temp = -pk_def / (pkwater_equiv * 1.27)
                    #               ! All the net rain freezes and becomes pack ice
                    #               Pk_ice = Pk_ice + Net_rain
                    pk_ice = pk_ice + net_rain

                #            ! (1.1.3) Rain in excess of amount required to bring pack
                #             !         to isothermal...
                #             ELSE
                # 23456789112345678921234567893123456789

                else:
                    #               ! Heat deficit and temperature of the snowpack go to 0
                    #               Pk_def = 0.0
                    #               Pk_temp = 0.0
                    pk_def = 0.0
                    pk_temp = 0.0
                    #               ! The portion of net rain that brings the snowpack to
                    #               ! isothermal freezes
                    #               Pk_ice = Pk_ice + pndz
                    pk_ice = pk_ice + pndz
                    #               ! The rest of the net rain becomes free water in the
                    #               ! snowpack
                    #               ! Note that there cannot be previous Freeh2o because the
                    #               ! snowpack had a heat deficit (all water was ice) before
                    #               ! this condition was reached.
                    #               Freeh2o = Net_rain - pndz
                    freeh2o = net_rain - pndz
                    #               ! Calculate the excess heat per area added by the portion
                    #               ! of rain that does not bring the snowpack to isothermal
                    #               ! (using specific heat of water)
                    #               calpr = train*(Net_rain-pndz)*INCH2CM ! [cal/cm^2]
                    calpr = train * (net_rain - pndz) * INCH2CM
                    #               ! Add the new heat to the snow pack
                    #               ! (the heat in this excess rain will melt some of the
                    #               ! pack ice when the water cools to 0 degC)
                    #               CALL calin(calpr, Pkwater_equiv, Pk_def, Pk_temp, &
                    #      &                   Pk_ice, Freeh2o, Snowcov_area, Snowmelt, &
                    #      &                   Pk_depth, Pss, Pst, Iasw, Pk_den, Freeh2o_cap, Den_max, Ihru_gl)
                    print("      ppt_to_pack 4", pkwater_equiv, snowmelt, pk_den)
                    (
                        iasw,
                        freeh2o,
                        pkwater_equiv,
                        pk_def,
                        pk_temp,
                        pk_ice,
                        pk_den,
                        snowmelt,
                        pss,
                        pst,
                        pk_depth,
                    ) = calin(
                        calpr,
                        pkwater_equiv,
                        pk_def,
                        pk_temp,
                        pk_ice,
                        freeh2o,
                        snowcov_area,
                        snowmelt,
                        pk_depth,
                        pss,
                        pst,
                        iasw,
                        pk_den,
                        freeh2o_cap,
                        den_max,
                        ihru_gl,
                    )
                    print("      ppt_to_pack 5", pkwater_equiv, snowmelt)
            #             ENDIF

            #           ! (1.2) Rain on snowpack that is isothermal
            #           !       at 0 degC (no heat deficit)...
            #           ! on glacier ice not active_layer goes in here only, as Pk_def, pndz = 0,
            #           ELSE
            # 23456789112345678921234567893123456789
            else:
                #             ! All net rain is added to free water in the snowpack
                #             Freeh2o = Freeh2o + Net_rain
                #             ! Calculate the heat per area added by the rain
                #             ! (using specific heat of water)
                #             calpr = train*Net_rain*INCH2CM ! [cal/cm^2]
                calpr = train * net_rain * INCH2CM
                #             ! Add the new heat to the snow pack
                #             ! (the heat in rain will melt some of the pack ice when
                #             ! the water cools to 0 degC)
                #             CALL calin(calpr, Pkwater_equiv, Pk_def, Pk_temp, &
                #      &                 Pk_ice, Freeh2o, Snowcov_area, Snowmelt, &
                #      &                 Pk_depth, Pss, Pst, Iasw, Pk_den, Freeh2o_cap, Den_max, Ihru_gl)
                print("      ppt_to_pack 6", pkwater_equiv, snowmelt)
                (
                    iasw,
                    freeh2o,
                    pkwater_equiv,
                    pk_def,
                    pk_temp,
                    pk_ice,
                    pk_den,
                    snowmelt,
                    pss,
                    pst,
                    pk_depth,
                ) = calin(
                    calpr,
                    pkwater_equiv,
                    pk_def,
                    pk_temp,
                    pk_ice,
                    freeh2o,
                    snowcov_area,
                    snowmelt,
                    pk_depth,
                    pss,
                    pst,
                    iasw,
                    pk_den,
                    freeh2o_cap,
                    den_max,
                    ihru_gl,
                )
                print("      ppt_to_pack 7", pkwater_equiv, snowmelt)
    #           ENDIF
    #         ENDIF

    #       ! (2) If there is net rain but no snowpack, set flag for a mix
    #       !     on no snowpack.
    #       ELSEIF ( Net_rain>0.0 ) THEN
    # 23456789112345678921234567893123456789

    elif net_rain > 0.0:
        print("      ppt_to_pack 8", pkwater_equiv, net_rain)
        #         ! Be careful with the code here.
        #         ! If this subroutine is called when there is an all-rain day
        #         ! on no existing snowpack (currently, it will not),
        #         ! then the flag here will be set inappropriately.
        #         Pptmix_nopack = ACTIVE ! [flag]
        pptmix_nopack = True
    #       ENDIF

    #       ! At this point, the subroutine has handled all conditions
    #       ! where there is net rain, so if there is net snow
    #       ! (doesn't matter if there is a pack or not)...
    #       IF ( Net_snow>0.0 ) THEN
    # 23456789112345678921234567893123456789
    print("      ppt_to_pack 9", pkwater_equiv, net_snow)
    if net_snow > 0.0:
        #         ! add the new snow to the pack water equivalent, precip, and ice
        #         Pkwater_equiv = Pkwater_equiv + DBLE(Net_snow)
        #         Pk_precip = Pk_precip + Net_snow
        #         Pk_ice = Pk_ice + Net_snow
        pkwater_equiv = pkwater_equiv + net_snow
        pk_precip = pk_precip + net_snow
        pk_ice = pk_ice + net_snow

        #         ! The temperature of the new snow will determine its effect on
        #         ! snowpack heat deficit
        #         ! 2 options below (if-then, else)

        #         ! (1) if the new snow is at 0 degC...
        #         IF ( tsnow>=0.0 ) THEN
        print("      ppt_to_pack 10", tsnow, pkwater_equiv, pk_precip, pk_ice, pk_temp)
        if tsnow >= 0.0:
            #           ! incoming snow does not change the overall heat content of
            #           ! the snowpack.
            #           ! However, the temperature will change, because the total heat
            #           ! content of the snowpack will be "spread out" among
            #           ! more snow.  Calculate the snow pack temperature from the
            #           ! heat deficit, specific heat of snow,
            #           ! and the new total snowpack water content
            #           Pk_temp = -Pk_def/SNGL(Pkwater_equiv*1.27D0) ! [degrees C]
            pk_temp = -pk_def / pkwater_equiv * 1.27
            print("      ppt_to_pack 11", pk_temp)

        #         ! (2) if the new snow is colder than 0 degC...
        #         ELSE
        else:
            #           ! calculate the amount of heat the new snow will absorb if
            #           ! warming it to 0C (negative number).
            #           ! This is the negative of the heat deficit of the new snow.
            #           calps = tsnow*Net_snow*1.27 ! [cal/cm^2]
            calps = tsnow * net_snow * 1.27
            #           ! The heat to warm the new snow can come from different
            #           ! sources depending on the state of the snowpack
            #           ! 2 options below (if-then, else)

            #           ! (2.1) if there is free water in the pack
            #           !       (at least some of it is going to freeze)...
            #           IF ( Freeh2o>0.0 ) THEN
            if freeh2o > 0.0:
                #             CALL caloss(calps, Pkwater_equiv, Pk_def, Pk_temp, Pk_ice, Freeh2o, Ihru_gl)
                print("      ppt_to_pack 12", pk_temp)
                pkwater_equiv, pk_def, pk_ice, freeh2o, pk_temp = caloss(
                    calps, pkwater_equiv, pk_def, pk_temp, pk_ice, freeh2o, ihru_gl
                )
                print("      ppt_to_pack 13", pk_temp)

            #           ! (2.2) if there is no free water (snow pack has a
            #           !       heat deficit greater than or equal to 0)...
            #           ELSE
            else:
                #             ! heat deficit increases because snow is colder than
                #             ! pack (minus a negative number = plus)
                #             ! and calculate the new pack temperature
                #             Pk_def = Pk_def - calps ! [cal/cm^2]
                #             Pk_temp = -Pk_def/SNGL(Pkwater_equiv*1.27D0) ! [degrees C]
                pk_def = pk_def - calps
                pk_temp = -pk_def / pkwater_equiv * 1.27
                print(
                    "      ppt_to_pack 14.0",
                    pk_def / pkwater_equiv,
                    pk_def / pkwater_equiv * 1.27,
                )
                print("      ppt_to_pack 14", pk_temp, pk_def, pkwater_equiv)
    #           ENDIF
    #         ENDIF
    #       ENDIF

    #       END SUBROUTINE ppt_to_pack
    print("      ppt_to_pack 15", pkwater_equiv, snowmelt)
    return (
        iasw,
        pptmix_nopack,
        snowmelt,
        freeh2o,
        pk_precip,
        pk_def,
        pk_ice,
        pk_den,
        snowcov_area,
        pk_temp,
        pkwater_equiv,
        pk_depth,
        pst,
        pss,
    )

In [ ]:
# !***********************************************************************
# !     Interpolate along snow covered area depletion curve
# !***********************************************************************
#   SUBROUTINE sca_deplcrv(Snowcov_area, Snarea_curve, Frac_swe)


def sca_deplcrv(snarea_curve, frac_swe):
    #  IF ( Frac_swe>1.0 ) THEN
    #     Snowcov_area = Snarea_curve(11)
    #   ELSE
    if frac_swe > 1.0:
        snowcov_area = snarea_curve[11 - 1]
    else:
        #     ! get the indices (as integers) of the depletion curve that
        #     ! bracket the given Frac_swe (next highest and next lowest)
        #     idx = INT( 10.0*(Frac_swe+0.2) ) ! [index]
        #     jdx = idx - 1 ! [index]
        idx = int(10.0 * (frac_swe + 0.2))
        jdx = idx - 1
        #     IF ( idx>11 ) idx = 11
        if idx > 11:
            idx = 11
        #     ! calculate the fraction of the distance (from the next lowest)
        #     ! the given Frac_swe is between the next highest and lowest
        #     ! curve values
        #     af = FLOAT( jdx-1 )
        #     dify = (Frac_swe*10.0) - af ! [fraction]
        af = float(jdx - 1)
        dify = (frac_swe * 10.0) - af
        #     ! calculate the difference in snow covered area represented
        #     ! by next highest and lowest curve values
        #     difx = Snarea_curve(idx) - Snarea_curve(jdx)
        difx = snarea_curve[idx - 1] - snarea_curve[jdx - 1]
        #     ! linearly interpolate a snow covered area between those
        #     ! represented by the next highest and lowest curve values
        #     Snowcov_area = Snarea_curve(jdx) + dify*difx
        snowcov_area = snarea_curve[jdx - 1] + dify * difx
    #   ENDIF
    #   END SUBROUTINE sca_deplcrv

    # ! Arguments
    # REAL, INTENT(OUT) :: Snowcov_area
    # REAL, INTENT(IN) :: Snarea_curve(11), Frac_swe

    return snowcov_area

In [ ]:
# !***********************************************************************
# !      Subroutine to compute snow-covered area
# !***********************************************************************
#   SUBROUTINE snowcov(Iasw, Newsnow, Snowcov_area, Snarea_curve, &
#  &                   Pkwater_equiv, Pst, Snarea_thresh, Net_snow, &
#  &                   Scrv, Pksv, Snowcov_areasv, Ai, Frac_swe)


def snowcov(
    iasw,
    newsnow,
    snowcov_area,
    snarea_curve,
    pkwater_equiv,
    pst,
    snarea_thresh,
    net_snow,
    scrv,
    pksv,
    snowcov_areasv,
):
    # ! Arguments
    #   INTEGER, INTENT(IN) :: Newsnow
    #   INTEGER, INTENT(INOUT) :: Iasw
    #   REAL, INTENT(IN) :: Snarea_thresh, Net_snow, Snarea_curve(11)
    #   DOUBLE PRECISION, INTENT(IN) :: Pkwater_equiv
    #   REAL, INTENT(INOUT) :: Snowcov_area
    #   DOUBLE PRECISION, INTENT(OUT) :: Ai
    #   REAL, INTENT(INOUT) :: Snowcov_areasv
    #   DOUBLE PRECISION, INTENT(INOUT) :: Pst, Scrv, Pksv
    #   REAL, INTENT(OUT) :: Frac_swe

    #   snowcov_area_ante = Snowcov_area
    snowcov_area_ante = snowcov_area
    #   ! Reset snowcover area to the maximum
    #   Snowcov_area = Snarea_curve(11) ! [fraction of area]
    snowcov_area = snarea_curve[11 - 1]

    #   ! Track the maximum pack water equivalent for the current
    #   ! snow pack
    #   IF ( Pkwater_equiv>Pst ) Pst = Pkwater_equiv ! [inches]
    if pkwater_equiv > pst:
        pst = pkwater_equiv

    #   ! Set ai to the maximum packwater equivalent, but no higher than
    #   ! the threshold for complete snow cover
    #   Ai = Pst ! [inches]
    ai = pst
    #   IF ( Ai>Snarea_thresh ) Ai = DBLE( Snarea_thresh ) ! [inches]
    if ai > snarea_thresh:
        ai = snarea_thresh

    #   ! calculate the ratio of the current packwater equivalent to
    #   ! the maximum packwater equivalent for the given snowpack
    #   IF ( Ai>DNEARZERO ) THEN
    if ai > 0.0:
        #     Frac_swe = SNGL( Pkwater_equiv/Ai ) ! [fraction]
        #     Frac_swe = MIN( 1.0, Frac_swe )
        frac_swe = pkwater_equiv / ai
        frac_swe = min(1.0, frac_swe)
    #   ELSE
    else:
        # !        print *, ai, snarea_thresh
        #     Frac_swe = 0.0
        frac_swe = 0.0
    #   ENDIF

    #   ! There are 3 potential conditions for the snow area curve:
    #   ! A. snow is accumulating and the pack is currently at its
    #   !    maximum level
    #   ! B. snow is depleting and the area is determined by the
    #   !    snow area curve
    #   ! C. new snow has occured on a depleting pack, temporarily
    #   !    resetting to 100% cover.
    #   ! For case (C), the snow covered area is linearly interpolated
    #   ! between 100% and the snow covered area before the new snow.
    #   ! In general, 1/4 of the new snow has to melt before the snow
    #   ! covered area goes below 100%, and then the remaining 3/4 has
    #   ! to melt to return to the previous snow covered area.

    #   ! First, the code decides whether snow is accumulating (A)
    #   ! or not (B/C).
    #   ! 2 options below (if-then, else)

    #   ! (1) The pack water equivalent is at the maximum
    #   IF ( Pkwater_equiv>=Ai ) THEN
    if pkwater_equiv >= ai:
        #     ! Stay on the snow area curve (it will be at the maximum
        #     ! because the pack water equivalent is equal to ai
        #     ! and it can't be higher)
        #     Iasw = 0
        iasw = 0

    #   ! (2) The pack water equivalent is less than the maximum
    #   ELSE
    else:
        #     ! If the snowpack isn't accumulating to a new maximum,
        #     ! it is either on the curve (condition B above) or being
        #     ! interpolated between the previous place on the curve and
        #     ! 100% (condition C above)
        #     ! 2 options below (if-then, elseif)

        #     ! (2.1) There was new snow...
        #     IF ( Newsnow/=0 ) THEN
        # 23456789112345678921234567893123456789
        if newsnow == True:
            #       ! New snow will always reset the snow cover to 100%.
            #       ! However, different states changes depending  on whether
            #       ! the previous snow area condition was on the curve or
            #       ! being interpolated between the curve and 100%
            #       ! 2 options below (if-then, else)

            #       ! (2.1.1) The snow area is being interpolated between 100%
            #       !         and a previous location on the curve...
            #       IF ( Iasw>0 ) THEN
            if iasw > 0.0:
                #         ! The location on the interpolated line is based on how
                #         ! much of the new snow has melted.  Because the first 1/4
                #         ! of the new snow doesn't matter, it has to keep track of
                #         ! the current snow pack plus 3/4 of the new snow.
                #         Scrv = Scrv + (0.75D0*DBLE(Net_snow)) ! [inches]
                scrv = scrv + (0.75 * net_snow)
            #         ! Scrv = Pkwater_equiv - (0.25D0*DBLE(Net_snow))) ! [inches]
            # !RAPCOMMENT - CHANGED TO INCREMENT THE SCRV VALUE IF ALREADY
            # !             INTERPOLATING BETWEEN CURVE AND 100%

            #   ! (2.1.2) The current snow area is on the curve...
            #   ELSE
            else:
                #     ! If switching from the snow area curve to interpolation
                #     ! between the curve and 100%, the current state of the snow
                #     ! pack has to be saved so that the interpolation can
                #     ! continue until back to the original conditions.
                #     ! First, set the flag to indicate interpolation between 100%
                #     ! and the previous area should be done
                #     Iasw = 1 ! [flag]
                iasw = 1
                #     ! Save the current snow covered area
                #     ! (before the new net snow)
                #     Snowcov_areasv = snowcov_area_ante ! [inches]
                snowcov_areasv = snowcov_area_ante
                #     ! Save the current pack water equivalent
                #     ! (before the new net snow)
                #     Pksv = Pkwater_equiv - DBLE( Net_snow ) ! [inches]
                pksv = pkwater_equiv - net_snow
                #     ! The location on the interpolated line is based on how much
                #     ! of the new snow has melted.  Because the first 1/4
                #     ! of the new snow doesn't matter, it has to keep track of
                #     ! the current snow pack plus 3/4 of the new snow.
                #     Scrv = Pkwater_equiv - (0.25D0*DBLE(Net_snow)) ! [inches]
                scrv = pkwater_equiv - (0.25 * net_snow)
            #   ENDIF
            #   ! The subroutine terminates here because the snow covered area
            #   ! always starts at 100% if there is any new snow (no need to
            #   ! reset it from the maximum value set at the beginning of the
            #   ! subroutine).
            #   RETURN
            return iasw, snowcov_area, ai, snowcov_areasv, pst, scrv, pksv, frac_swe

        # ! (2.2) There was no new snow, but the snow covered area is
        # !       currently being interpolated between 100%
        # !       from a previous new snow and the snow covered area
        # !       before that previous new snow...
        #     ELSEIF ( Iasw/=0 ) THEN
        # 23456789112345678921234567893123456789
        elif iasw == True:
            #       ! If the first 1/4 of the previous new snow has not melted,
            #       ! yet, then the snow covered area is still
            #       ! 100% and the subroutine can terminate.
            #       IF ( Pkwater_equiv>Scrv ) RETURN
            if pkwater_equiv > scrv:
                return iasw, snowcov_area, ai, snowcov_areasv, pst, scrv, pksv, frac_swe

            #       ! At this point, the program is almost sure it is
            #       ! interpolating between the previous snow covered area and
            #       ! 100%, but it is possible that enough snow has melted to
            #       ! return to the snow covered area curve instead.
            #       ! 2 options below (if-then, else)

            #       ! (2.2.1) The snow pack still has a larger water equivalent
            #       !         than before the previous new snow.  I.e., new snow
            #       !         has not melted back to original area...
            #       IF ( Pkwater_equiv>=Pksv ) THEN
            # 23456789112345678921234567893123456789
            if pkwater_equiv >= pksv:
                #         ! Do the interpolation between 100% and the snow covered
                #         ! area before the previous new snow.

                #         ! Calculate the difference between the maximum snow
                #         ! covered area (remember that Snowcov_area is always
                #         ! set to the maximum value at this point) and the snow
                #         ! covered area before the last new snow.
                #         difx = DBLE( Snowcov_area - Snowcov_areasv )
                difx = snowcov_area - snowcov_areasv
                #         ! Calculate the difference between the water equivalent
                #         ! before the last new snow and the previous water
                #         ! equivalent plus 3/4 of the last new snow.
                #         ! In effect, get the value of 3/4 of the previous
                #         ! new snow.
                #         dify = Scrv - Pksv ! [inches]                       !gl1098
                dify = scrv - pksv

                #         ! If 3/4 of the previous new snow is significantly
                #         ! different from zero, then calculate the ratio of the
                #         ! unmelted amount of previous new snow in the snow pack
                #         ! to the value of 3/4 of previous new snow.
                #         ! In effect, this is the fraction of the previous new snow
                #         ! that determines the current interpolation
                #         ! of snow covered area.
                #         fracy = 0.0D0 ! [fraction]                             !gl1098
                fracy = 0.0
                #         IF ( dify>0.0D0 ) fracy = (Pkwater_equiv-Pksv)/dify
                #                                                        ! [fraction]
                if dify > 0.0:
                    fracy = (pkwater_equiv - pksv) / dify
                #         ! Linearly interpolate the new snow covered area.
                #         Snowcov_area = Snowcov_areasv + SNGL(fracy*difx)
                #                                                ! [fraction of area]
                snowcov_area = snowcov_areasv + (fracy * difx)
                #         ! Terminate the subroutine
                #         RETURN
                return iasw, snowcov_area, ai, snowcov_areasv, pst, scrv, pksv, frac_swe

            #       ! (2.2.2) The snow pack has returned to the snow water
            #       ! equivalent before the previous new snow. I.e. back to
            #       ! original area before new snow.
            #       ELSE
            else:
                #         ! Reset the flag to use the snow area curve
                #         Iasw = 0 ! [flag]
                iasw = 0
        #       ENDIF

        #     ENDIF

        #     ! If this subroutine is still running at this point, then the
        #     ! program knows that the snow covered area needs to be
        #     ! adjusted according to the snow covered area curve.  So at
        #     ! this point it must interpolate between points on the snow
        #     ! covered area curve (not the same as interpolating between
        #     ! 100% and the previous spot on the snow area depletion curve).

        #     CALL sca_deplcrv(Snowcov_area, Snarea_curve, Frac_swe)
        sca_deplcrv(snowcov_area, snarea_curve, frac_swe)

    #   ENDIF

    #   END SUBROUTINE snowcov

    return iasw, snowcov_area, pst, scrv, pksv, snowcov_areasv, ai, frac_swe

In [ ]:
# !***********************************************************************
# !      Subroutine to compute snowpack albedo
# !***********************************************************************
# SUBROUTINE snalbedo(Newsnow, Iso, Lst, Snsv, Prmx, Pptmix, Albset_rnm, &
# &                    Net_snow, Albset_snm, Albset_rna, Albset_sna, Albedo, &
# &                    Int_alb, Salb, Slst)


def snalbedo(
    newsnow,
    iso,
    lst,
    snsv,
    prmx,
    pptmix,
    albset_rnm,
    net_snow,
    albset_snm,
    albset_rna,
    albset_sna,
    int_alb,
    salb,
    slst,
):
    # ! Arguments
    # INTEGER, INTENT(IN) :: Newsnow, Iso, Pptmix
    # INTEGER, INTENT(INOUT) :: Int_alb, Lst
    # REAL, INTENT(IN) :: Albset_rnm, Albset_snm, Albset_rna, Albset_sna, Prmx, Net_snow
    # REAL, INTENT(INOUT) :: Salb, Slst, Snsv
    # REAL, INTENT(OUT) :: Albedo

    albedo = 0.0

    #       ! The albedo is always reset to a new initial (high) value when
    #       ! there is new snow above a threshold (parameter).  Albedo
    #       ! is then a function of the number of days since the last new snow
    #       ! Intermediate conditions apply when there is new snow
    #       ! below the threshold to reset the albedo to its highest value.
    #       ! The curve for albedo change (decreasing) is different for the
    #       ! snow accumulation season and the snow melt season.
    #       ! The albedo first depends on if there is no new snow during the
    #       ! current time step, if there is new snow during accumulation
    #       ! season, or if there is new snow during melt season.
    #       ! 3 options below (if-then, elseif, else)

    #       ! (1) There is no new snow
    #       IF ( Newsnow==OFF ) THEN
    # ---1---2---3---4
    if newsnow is False:
        #         ! If no new snow, check if there was previous new snow that
        #         ! was not sufficient to reset the albedo (Lst=1)
        #         ! Lst can only be greater than 0 during melt season (see below)
        #         IF ( Lst>0 ) THEN
        # ---1---2---3---4
        if lst > 0:
            #           ! Slst is the number of days (float) since the last
            #           ! new snowfall
            #           ! Set the albedo curve back three days from the number
            #           ! of days since the previous snowfall
            #           ! (see Salb assignment below)
            #           ! (note that "shallow new snow" indicates new snow that
            #           ! is insufficient to completely reset the albedo curve)
            #           ! In effect, a shallow new snow sets the albedo curve back
            #           ! a few days, rather than resetting it entirely.
            #           Slst = Salb - 3.0 ! [days]
            # ---1---2---3---4---5
            slst = salb - 3.0
            #           ! Make sure the number of days since last new snow
            #           ! isn't less than 1
            #           IF ( Slst<1.0 ) Slst = 1.0 ! [days]
            # ---1---2---3---4---5
            if slst < 1.0:
                slst = 1.0
            #           ! If not in melt season
            #           IF ( Iso/=2 ) THEN
            if iso != 2:
                #             ! Note that this code is unreachable in its current state.
                #             ! This code is only run during melt season due to the
                #             ! fact that Lst can only be set to 1 in the melt season.
                #             ! Therefore, Iso is always going to be equal to 2.
                #             ! Make sure the maximum point on the albedo curve is 5
                #             ! In effect, if there is any new snow, the albedo can
                #             ! only get so low in accumulation season, even if the
                #             ! new snow is insufficient to reset albedo entirely
                #             IF ( Slst>5.0 ) Slst = 5.0 ! [days]
                # ---1---2---3---4---5
                if slst > 5.0:
                    slst = 5.0

            #           ENDIF
            #           ! Reset the shallow new snow flag and cumulative shallow
            #           ! snow variable (see below)
            #           Lst = 0 ! [flag]
            #           Snsv = 0.0 ! [inches]
            # ---1---2---3---4---5
            lst = 0
            snsv = 0.0
        #         ENDIF

        #       ! (2) New snow during the melt season
        #       ELSEIF ( Iso==2 ) THEN
        # ---1---2---3---4---5
        elif iso == 2:
            # ! RAPCOMMENT - CHANGED TO ISO FROM MSO

            #         ! If there is too much rain in a precipitation mix,
            #         ! albedo will not be reset
            #         ! New snow changes albedo only if the fraction rain
            #         ! is less than the threshold above which albedo is not reset
            #         IF ( Prmx<Albset_rnm ) THEN
            # ---1---2---3---4---5
            if prmx < albset_rnm:
                #           ! If the fraction rain doesn't prevent the albedo from
                #           ! being reset, then how the albedo changes depends on
                #           ! whether the snow amount is above or below the threshold
                #           ! for resetting albedo
                #           ! 2 options below (if-then, else)

                #           ! (2.1) If there is enough new snow to reset the albedo
                #           IF ( Net_snow>Albset_snm ) THEN
                # ---1---2---3---4---5
                if net_snow > albset_snm:
                    #             ! Reset number of days since last new snow to 0
                    #             Slst = 0.0 ! [days]
                    #             Lst = 0 ! [flag]
                    slst = 0.0
                    lst = 0
                    #             ! Reset the saved new snow to 0
                    #             Snsv = 0.0 ! [inches]
                    snsv = 0.0

                #           ! (2.2) If there is not enough new snow this time period
                #           ! to reset the albedo on its own
                #           ELSE
                else:
                    #             ! Snsv tracks the amount of snow that has fallen as long
                    #             ! as the total new snow is not
                    #             ! enough to reset the albedo.
                    #             Snsv = Snsv + Net_snow ! [inches]
                    snsv = snsv + set_snow

                    #             ! Even if the new snow during this time period is
                    #             ! insufficient to reset the albedo, it may still reset the
                    #             ! albedo if it adds enough to previous shallow snow
                    #             ! accumulation.  The change in Albedo depends on if the
                    #             ! total amount of accumulated shallow snow has become enough
                    #             ! to reset the albedo or not.
                    #             ! 2 options below (if-then, else)

                    #             ! (2.2.1) If accumulated shallow snow is enough to reset
                    #             !         the albedo
                    #             IF ( Snsv>Albset_snm ) THEN
                    # ---1---2---3---4---5
                    if snsv > albset_snm:
                        #               ! Reset the albedo states.
                        #               Slst = 0.0 ! [days]
                        #               Lst = 0 ! [flag]
                        #               Snsv = 0.0 ! [inches]
                        slst = 0.0
                        lst = 0
                        snsv = 0.0

                    #             ! (2.2.2) If the accumulated shallow snow is not enough to
                    #             !         reset the albedo curve
                    #             ELSE
                    else:
                        #               ! Salb records the number of days since the last new snow
                        #               ! that reset albedo
                        #               IF ( Lst==0 ) Salb = Slst ! [days]
                        # ---1---2---3---4---5---6
                        if lst == 0:
                            #               ! Reset the number of days since new snow
                            #               Slst = 0.0 ! [days]
                            slst = 0.0
                            #               ! set the flag indicating that there is shallow new snow
                            #               ! (i.e. not enough new snow to reset albedo)
                            #               Lst = 1 ! [flag]
                            lst = 1
            #             ENDIF
            #           ENDIF
            #         ENDIF
            #       ! (3) New snow during the accumulation season
            #       ELSE
            # ---1---2---3---4---5---6
            else:
                #         ! The change in albedo depends on if the precipitation is a mix,
                #         ! if the rain is above a threshold,  or if the snow is above
                #         ! a threshold.
                #         ! 4 options below (if-then, elseif, elseif, else)

                #         ! (3.1) If it is not a mixed event...
                #         IF ( Pptmix==OFF ) THEN
                # ---1---2---3---4---5---6
                if pptmix is False:
                    #           ! During the accumulation season, the threshold for resetting
                    #           ! the albedo does not apply if there is a snow-only event.
                    #           ! Therefore, no matter how little snow there is, it will
                    #           ! always reset the albedo curve the the maximum, if it
                    #           ! occurs during the accumulation season.
                    #           ! reset the time since last snow to 0
                    #           Slst = 0.0 ! [days]
                    # ---1---2---3---4---5---6
                    slst = 0.0
                    #           ! there is no new shallow snow
                    #           Lst = 0 ! [flag]
                    lst = 0.0

                #         ! (3.2) If it is a mixed event and the fraction rain is above
                #         !       the threshold above which albedo is not reset...
                #         ELSEIF ( Prmx>=Albset_rna ) THEN
                # ---1---2---3---4---5---6
                elif prmx >= albset_rna:
                    #           ! there is no new shallow snow
                    #           Lst = 0 ! [flag]
                    lst = 0
                #           ! albedo continues to decrease on the curve

                #         ! (3.3) If it is a mixed event and there is enough new snow
                #         !       to reset albedo...
                #         ELSEIF ( Net_snow>=Albset_sna ) THEN
                elif net_snow >= albset_sna:
                    #           ! reset the albedo
                    #           Slst = 0.0 ! [days]
                    #           ! there is no new shallow snow
                    #           Lst = 0 ! [flag]
                    slst = 0.0
                    lst = 0

                #         ! (3.4) If it is a mixed event and the new snow was not
                #         !       enough to reset the albedo...
                #         ELSE
                # ---1---2---3---4---5---6
                else:
                    #           ! set the albedo curve back 3 days (increasing the albedo)
                    #           Slst = Slst - 3.0 ! [days]
                    slst = slst - 3.0
                    #           ! Make sure the number of days since last new snow is not
                    #           ! less than 0
                    #           IF ( Slst<0.0 ) Slst = 0.0 ! [days]
                    if slst < 0.0:
                        slst = 0.0
                    #           ! Make sure the number of days since last new snow is not
                    #           ! greater than 5
                    #           ! In effect, if there is any new snow, the albedo can
                    #           ! only get so low in accumulation season, even if the
                    #           ! new snow is insufficient to reset albedo entirely
                    #           IF ( Slst>5.0 ) Slst = 5.0 ! [days]
                    #           Lst = 0 ! [flag]
                    if slst > 5.0:
                        slst = 5.0
                    lst = 0
                #         ENDIF
                #         Snsv = 0.0 ! [inches]
                snsv = 0.0
            #       ENDIF
            #       ! At this point, the subroutine knows where on the curve the
            #       ! albedo should be based on current conditions and the
            #       ! new snow (determined by value of Slst variable)

            #       ! Get the integer value for days (or effective days)
            #       ! since last snowfall
            #       l = INT(Slst+0.5) ! [days]
            # ---1---2---3---4---5---6
            l = int(slst + 0.5)

            #       ! Increment the state variable for days since the
            #       ! last snowfall
            #       Slst = Slst + 1.0 ! [days]
            slst = slst + 1.0

            #       !******Compute albedo
            #       ! Albedo will only be different from the max (default value)
            #       ! if it has been more than 0 days since the last new snow
            #       ! capable of resetting the albedo.  If albedo is at the
            #       ! maximum, the maximum is different for accumulation and
            #       ! melt season.
            #       ! 3 options below (if-then, elseif, else)

            #       ! (1) It has been more than 0 days since the last new snow
            #        IF ( l>0 ) THEN
            # ---1---2---3---4---5---6
            if l > 0:
                #         ! Albedo depends on whether it is currently on the
                #         ! accumulation season curve or on the melt season curve.
                #         ! 3 options below (if-then, elseif, else)

                #         ! (1.1) Currently using the melt season curve
                #         !       (Old snow - Spring melt period)...
                #         IF ( Int_alb==2 ) THEN
                # ---1---2---3---4---5---6
                if int_alb == 2:
                    #           ! Don't go past the last possible albedo value
                    #           IF ( l>MAXALB ) l = MAXALB ! [days]
                    if l > MAXALB:
                        l = MAXALB
                    #           ! Get the albedo number from the melt season curve
                    #           Albedo = Amlt(l) ! [fraction of radiation]
                    albedo = amlt[l - 1]

                #         ! (1.2) Currently using the accumulation season curve
                #         !       (Old snow - Winter accumulation period)...
                #         ! and not past the maximum curve index
                #         ELSEIF ( l<=MAXALB ) THEN
                elif l <= MAXALB:
                    #           ! Get the albedo number from the accumulation season curve
                    #           Albedo = Acum(l) ! [fraction of radiation]
                    albedo = acum[l - 1]

                #         ! (1.3) Currently using the accumulation season curve and
                #         !       past the maximum curve index...
                #         ELSE
                else:
                    #           ! start using the the MELT season curve at 12 days
                    #           ! previous to the current number of days since the last
                    #           ! new snow
                    #           l = l - 12 ! [days]
                    l = l - 12
                    #           ! keep using the melt season curve until its minimum
                    #           ! value (maximum index) is reached or until there is new snow
                    #           IF ( l>MAXALB ) l = MAXALB ! [days]
                    if l > MAXALB:
                        l = MAXALB
                    #           ! get the albedo value from the melt season curve
                    #           Albedo = Amlt(l) ! [fraction of radiation]
                    albedo = amlt[l - 1]
            #         ENDIF

            #       ! (2) New snow has reset the albedo and it is melt season
            #       ELSEIF ( Iso==2 ) THEN
            elif iso == 2:
                # ! RAPCOMMENT - CHANGED TO ISO FROM MSO
                #         ! Set albedo to initial value during melt season
                #         Albedo = 0.72 ! [fraction of radiation] value Rob suggested
                albedo = 0.72
                # !       Albedo = 0.81 ! [fraction of radiation] original value
                #         ! Int_alb is a flag to indicate use of the melt season curve (2)
                #         ! or accumulation season curve (1)
                #         ! Set flag to indicate melt season curve
                #         Int_alb = 2 ! [flag]
                int_alb = 2

            #       ! (3) New snow has reset the albedo and it is accumulation season
            #       ELSE
            else:
                #         ! Set albedo to initial value during accumulation season
                #         Albedo = 0.91 ! [fraction of radiation]
                albedo = 0.91
                #         ! Set flag to indicate accumulation season curve
                #         Int_alb = 1 ! [flag]
                int_alb = 1
    #       ENDIF

    #       END SUBROUTINE snalbedo

    return int_alb, lst, salb, snsv, albedo

In [ ]:
def snowbal(
    niteda,
    tstorm_mo,
    iasw,
    temp,
    esv,
    hru_ppt,
    trd,
    emis_noppt,
    canopy_covden,
    cec,
    pkwater_equiv,
    pk_def,
    pk_temp,
    pk_ice,
    freeh2o,
    snowcov_area,
    snowmelt,
    pk_depth,
    pss,
    pst,
    pk_den,
    cst,
    sw,
    freeh2o_cap,
    den_max,
    ihru_gl,
):
    #     INTEGER, INTENT(IN) :: Niteda, Tstorm_mo, Ihru_gl
    #     INTEGER, INTENT(INOUT) :: Iasw
    #     REAL, INTENT(IN) :: Temp, Esv, Trd, Cec, Cst, Canopy_covden
    #     REAL, INTENT(IN) :: Emis_noppt, Sw, Freeh2o_cap
    #     REAL, INTENT(IN) :: Hru_ppt, Snowcov_area, Den_max
    #     DOUBLE PRECISION, INTENT(INOUT) :: Pst, Pss
    #     REAL, INTENT(OUT) :: Cal
    #     REAL, INTENT(INOUT) :: Pk_den, Pk_def, Pk_temp, Pk_ice
    #     REAL, INTENT(INOUT) :: Freeh2o, Snowmelt
    #     DOUBLE PRECISION, INTENT(INOUT) :: Pkwater_equiv, Pk_depth

    cal = 0.0

    return (
        iasw,
        pkwater_equiv,
        pk_def,
        pk_temp,
        pk_ice,
        freeh2o,
        snowmelt,
        pk_depth,
        pss,
        pst,
        pk_den,
        cal,
    )

In [ ]:
def snorun(
    dates,
    basin_horad,
    orad,
    newsnow,
    tmaxf,
    tminf,
    net_snow,
    net_rain,
    pptmix,
    prmx,
    net_ppt,
    hru_ppt,
    swrad,
    canopy_covden,
    transp_on,
    hru_area,
    hru_lat,
    melt_force,
    melt_look,
    tmax_allsnow,
    freeh2o_cap,
    den_max,
    hru_deplcrv,
    snarea_thresh,
    albset_rnm,
    albset_sna,
    albset_snm,
    albset_rna,
    cecn_coef,
    emis_noppt,
    cov_type,
    rad_trncf,
    den_init,
    settle_const,
    tstorm_mo,
):
    pkwater_equiv = np.zeros(len(dates))
    pss = np.zeros(len(dates))
    pst = np.zeros(len(dates))
    snowcov_area = np.zeros(len(dates))
    pk_def = np.zeros(len(dates))
    pk_temp = np.zeros(len(dates))
    pk_ice = np.zeros(len(dates))
    freeh2o = np.zeros(len(dates))
    iasw = np.zeros(len(dates), dtype=int)
    snowmelt = np.zeros(len(dates))
    pk_depth = np.zeros(len(dates))
    pk_den = np.zeros(len(dates))
    pptmix_nopack = np.zeros(len(dates), dtype=int)
    pk_precip = np.zeros(len(dates))
    pkwater_ante = np.zeros(len(dates))
    snow_evap = np.zeros(len(dates))
    tcal = np.zeros(len(dates))
    ai = np.zeros(len(dates))
    lst = np.zeros(len(dates))
    albedo = np.zeros(len(dates))
    freeh20 = np.zeros(len(dates))
    snsv = np.zeros(len(dates))
    scrv = np.zeros(len(dates))
    pksv = np.zeros(len(dates))
    snowcov_areasv = np.zeros(len(dates))
    frac_swe = np.zeros(len(dates))
    int_alb = np.zeros(len(dates), dtype=int)
    salb = np.zeros(len(dates))
    slst = np.zeros(len(dates))
    albedo = np.zeros(len(dates))
    iso = np.zeros(len(dates), dtype=int)
    mso = np.zeros(len(dates), dtype=int)
    lso = np.zeros(len(dates), dtype=int)

    hemisphere = which_hemisphere(hru_lat, hru_area)

    ihru = 0
    ii = 0
    for date in dates:
        jday = day_of_year(date)
        julwater = julian_day_water(hemisphere, jday)
        imon = date.month - 1
        print("snorun: ", date)

        #         ! Calculate the ratio of measured radiation to potential radiation
        #         ! (used as a cumulative indicator of cloud cover)

        #        basin_horad from ddsolrad and soltab_basinpotsw from solar table are the same thing
        #         but basin_horad is easier to get.
        trd = orad[ii] / basin_horad[ii]

        #         ! By default, the precipitation added to snowpack, snowmelt,
        #         ! and snow evaporation are 0
        frac_swe[ii] = 0.0
        pk_precip[ii] = 0.0
        snowmelt[ii] = 0.0
        snow_evap[ii] = 0.0
        tcal[ii] = 0.0
        ai[ii] = 0.0

        #         ! Keep track of the pack water equivalent before it is changed
        #         ! by precipitation during this time step
        if ii > 0:
            pkwater_ante[ii] = pkwater_equiv[ii - 1]
            pkwater_equiv[ii] = pkwater_equiv[ii - 1]
            pk_def[ii] = pk_def[ii - 1]
            pk_temp[ii] = pk_temp[ii - 1]
            pk_ice[ii] = pk_ice[ii - 1]
            snowcov_area[ii] = snowcov_area[ii - 1]
            pk_den[ii] = pk_den[ii - 1]
            iso[ii] = iso[ii - 1]
            mso[ii] = mso[ii - 1]
            lso[ii] = lso[ii - 1]

        print("snorun: pk_den = ", pk_den[ii])

        #         ! If it's the first julian day of the water year, several
        #         ! variables need to be reset
        #         ! - reset the previous snow water eqivalent plus new snow to 0
        #         ! - reset flags to indicate it is not melt season or potetential melt season
        #         ! - reset the counter for the number of days a snowpack is at 0 deg Celsius
        #         !rsr, do we want to reset all HRUs, what about Southern Hemisphere
        #         IF ( Julwater==1 ) THEN
        #             Pss(i) = 0.0D0 ! [inches]
        #             Iso(i) = 1 ! [flag]
        #             Mso(i) = 1 ! [flag]
        #             Lso(i) = 0 ! [counter]

        # 23456789112345678921234567893123456789
        if julwater == 1:
            pss[ii] = 0.0
            iso[ii] = 1
            mso[ihru] = 1
            lso[ihru] = 0

        #         ! HRU SET-UP - SET DEFAULT VALUES AND/OR BASE
        #         !              CONDITIONS FOR THIS TIME PERIOD
        #         !**************************************************************

        #         ! By default, there has not been a mixed event without a
        #         ! snowpack
        #         Pptmix_nopack(i) = OFF ! [flag]

        #         ! If the day of the water year is beyond the forced melt day
        #         ! indicated by the parameter, then set the flag indicating
        #         ! melt season
        #         !rsr, need to rethink this at some point
        # !rsr10  IF ( Iso(i)/=2 ) THEN
        #           IF ( Jday==Melt_force(i) ) Iso(i) = 2 ! [flag]
        # !rsr10  ENDIF

        #         ! If the day of the water year is beyond the first day to
        #         ! look for melt season indicated by the parameter,
        #         ! then set the flag indicating to watch for melt season
        #         !rsr, need to rethink this at some point
        # !rsr10  IF ( Mso(i)/=2 ) THEN
        #           IF ( Jday==Melt_look(i) ) Mso(i) = 2 ! [flag]
        # !rsr10  ENDIF

        #         ! Skip the HRU if there is no snowpack and no new snow and not a glacier
        #         IF ( Active_glacier==OFF ) THEN
        #           IF ( Pkwater_equiv(i)<DNEARZERO .AND. Newsnow(i)==0 ) THEN
        #             Snowcov_area(i) = 0.0 ! reset to be sure it is zero if snowpack melted on last timestep
        #             CYCLE
        #           ENDIF
        #         ENDIF

        # 23456789112345678921234567893123456789
        pptmix_nopack[ihru] = False
        if jday == melt_force[ihru]:
            iso[ihru] = 2
        if jday == melt_look[ihru]:
            mso[ihru] = 2

        if pkwater_equiv[ii - 1] == 0.0 and newsnow[ii - 1] == 0:
            snowcov_area[ii] = 0.0

        #         ! If there is no existing snow pack and there is new snow, the
        #         ! initial snow covered area is complete (1)
        #         IF ( Newsnow(i)==ACTIVE .AND. Pkwater_equiv(i)<DNEARZERO ) Snowcov_area(i) = 1.0 ! [fraction of area]
        #         IF ( Active_glacier==1 ) Glacrcov_area(i) =(1.0-Snowcov_area(i))*Glacier_frac(i)
        #         IF ( Active_glacier==2 ) Glacrcov_area(i) =(1.0-Snowcov_area(i))*Glrette_frac(i)

        #         ! HRU STEP 1 - DEAL WITH PRECIPITATION AND ITS EFFECT ON THE WATER
        #         !              CONTENT AND HEAT CONTENT OF SNOW PACK
        #         !************************************************************

        #         ! If there is net precipitation on an existing snowpack, OR if
        #         ! there is any net snow, add the incoming water (or ice) and
        #         ! heat (or heat deficit) to the snowpack
        #         IF ( (Pkwater_equiv(i)>0.0D0.AND.Net_ppt(i)>0.0) .OR. Net_snow(i)>0.0 ) &
        #      &       CALL ppt_to_pack(Pptmix(i), Iasw(i), Tmaxc(i), Tminc(i), Tavgc(i), &
        #      &                        Pkwater_equiv(i), Net_rain(i), Pk_def(i), &
        #      &                        Pk_temp(i), Pk_ice(i), Freeh2o(i), Snowcov_area(i), &
        #      &                        Snowmelt(i), Pk_depth(i), Pss(i), Pst(i), Net_snow(i), &
        #      &                        Pk_den(i), Pptmix_nopack(i), Pk_precip(i), Tmax_allsnow_c(i,Nowmonth), &
        #      &                        Freeh2o_cap(i), Den_max(i), not_a_glacier_hru)
        #         IF ( Active_glacier>OFF ) THEN
        #           IF ( Glacrcov_area(i)>0.0.AND.Glacr_pkwater_ante(i)>0.0D0.AND.Net_ppt(i)>0.0 &
        #      &         .AND.Pptmix(i)==0.AND.Net_snow(i)==0.0 ) THEN
        #              CALL ppt_to_pack(0, Iasw(i), Tmaxc(i), Tminc(i), Tavgc(i), &
        #      &                        Glacr_Pkwater_equiv(i), Net_rain(i), Glacr_pk_def(i), &
        #      &                        Glacr_pk_temp(i), Glacr_pk_ice(i), Glacr_freeh2o(i), Glacrcov_area(i), &
        #      &                        Glacrmelt(i), Glacr_pk_depth(i), Glacr_pss(i), Glacr_pst(i), 0.0, &
        #      &                        Glacr_pk_den(i), Pptmix_nopack(i), Pk_precip(i), Tmax_allsnow_c(i,Nowmonth), &
        #      &                        Glacr_freeh2o_capm(i), Den_max(i), i)
        #           ENDIF
        #           ! FOLLOWING does basal melt on glacier
        #           ! Paterson 2010 says 12 mm/yr for friction and geothermal heating
        #           IF ( Active_glacier==1 ) Glacrb_melt(i) = 12.0*0.03937/DAYS_YR*Glacier_frac(i)
        #           IF ( Active_glacier==2 ) Glacrb_melt(i) = 12.0*0.03937/DAYS_YR*Glrette_frac(i) !since not moving much, maybe =0
        #         ENDIF

        # 23456789112345678921234567893123456789
        if newsnow[ihru] == True and pkwater_equiv[ii] == 0.0:
            snowcov_area[ii] = 1.0

        if (pkwater_equiv[ii] > 0.0 and net_ppt[ii] > 0.0) or net_snow[ii] > 0.0:
            tmax_allsnow_c = (tmax_allsnow[imon, ihru] - 32.0) * 5.0 / 9.0
            tmaxc = (tmaxf[ii] - 32.0) * 5.0 / 9.0
            tminc = (tminf[ii] - 32.0) * 5.0 / 9.0
            tavgc = (tmaxc + tminc) / 2.0
            print("   snorun: ppt_to_pack calling ppt_to_pack 1")
            print(
                "   snorun: 0.5",
                pptmix[ii],
                iasw[ii],
                tmaxc,
                tminc,
                tavgc,
                pkwater_equiv[ii],
                net_rain[ii],
                pk_def[ii],
                pk_temp[ii],
                pk_ice[ii],
                freeh2o[ii],
                snowcov_area[ii],
                snowmelt[ii],
                pk_depth[ii],
                pss[ii],
                pst[ii],
                net_snow[ii],
                pk_den[ii],
                pptmix_nopack[ii],
                pk_precip[ii],
                tmax_allsnow_c,
                freeh2o_cap[ihru],
                den_max[ihru],
                NOT_A_GLACIER_HRU,
            )
            print(
                "   snorun: 0.6: pkwater_equiv =",
                pkwater_equiv[ii],
                "snowmelt =",
                snowmelt[ii],
                "snow_evap =",
                snow_evap[ii],
            )
            print("   snorun: 0.6.1: pk_den =", pk_den[ii])
            (
                iasw[ii],
                pptmix_nopack[ii],
                snowmelt[ii],
                freeh2o[ii],
                pk_precip[ii],
                pk_def[ii],
                pk_ice[ii],
                pk_den[ii],
                snowcov_area[ii],
                pk_temp[ii],
                pkwater_equiv[ii],
                pk_depth[ii],
                pst[ii],
                pss[ii],
            ) = ppt_to_pack(
                pptmix[ii],
                iasw[ii],
                tmaxc,
                tminc,
                tavgc,
                pkwater_equiv[ii],
                net_rain[ii],
                pk_def[ii],
                pk_temp[ii],
                pk_ice[ii],
                freeh2o[ii],
                snowcov_area[ii],
                snowmelt[ii],
                pk_depth[ii],
                pss[ii],
                pst[ii],
                net_snow[ii],
                pk_den[ii],
                pptmix_nopack[ii],
                pk_precip[ii],
                tmax_allsnow_c,
                freeh2o_cap[ihru],
                den_max[ihru],
                NOT_A_GLACIER_HRU,
            )
            print(
                "   snorun: 0.7: pkwater_equiv =",
                pkwater_equiv[ii],
                "snowmelt =",
                snowmelt[ii],
                "snow_evap =",
                snow_evap[ii],
            )
            print("   snorun: 0.7.1: pk_den =", pk_den[ii])

        #    IF ( Active_glacier>OFF ) THEN
        #       IF ( Glacrcov_area(i)>0.0.AND.Glacr_pkwater_ante(i)>0.0D0.AND.Net_ppt(i)>0.0 &
        #  &         .AND.Pptmix(i)==0.AND.Net_snow(i)==0.0 ) THEN
        #          CALL ppt_to_pack(0, Iasw(i), Tmaxc(i), Tminc(i), Tavgc(i), &
        #  &                        Glacr_Pkwater_equiv(i), Net_rain(i), Glacr_pk_def(i), &
        #  &                        Glacr_pk_temp(i), Glacr_pk_ice(i), Glacr_freeh2o(i), Glacrcov_area(i), &
        #  &                        Glacrmelt(i), Glacr_pk_depth(i), Glacr_pss(i), Glacr_pst(i), 0.0, &
        #  &                        Glacr_pk_den(i), Pptmix_nopack(i), Pk_precip(i), Tmax_allsnow_c(i,Nowmonth), &
        #  &                        Glacr_freeh2o_capm(i), Den_max(i), i)
        #       ENDIF

        #     ! If there is still a snowpack
        #     IF ( Pkwater_equiv(i)>0.0D0 ) THEN
        # 23456789112345678921234567893123456789
        if pkwater_equiv[ii] > 0.0:
            #       ! HRU STEP 2 - CALCULATE THE NEW SNOW COVERED AREA
            #       !**********************************************************
            #       ! Compute snow-covered area from depletion curve
            #       k = Hru_deplcrv(i)
            k = hru_deplcrv[ihru] - 1
            #       ! calculate the new snow covered area
            #       CALL snowcov(Iasw(i), Newsnow(i), Snowcov_area(i), &
            #  &                 Snarea_curve(1, k), Pkwater_equiv(i), Pst(i), &
            #  &                 Snarea_thresh(i), Net_snow(i), Scrv(i), &
            #  &                 Pksv(i), Snowcov_areasv(i), Ai(i), Frac_swe(i))
            #             print(iasw.shape, newsnow.shape, snowcov_area.shape, snarea_curve.shape, pkwater_equiv.shape)
            #             print(        pst.shape, snarea_thresh.shape, net_snow.shape, scrv.shape, pksv.shape, snowcov_areasv.shape)
            #             print(        ai.shape, frac_swe.shape)
            (
                iasw[ii],
                snowcov_area[ii],
                pst[ii],
                scrv[ii],
                pksv[ii],
                snowcov_areasv[ii],
                ai[ii],
                frac_swe[ii],
            ) = snowcov(
                iasw[ii],
                newsnow[ii],
                snowcov_area[ii],
                snarea_curve,
                pkwater_equiv[ii],
                pst[ii],
                snarea_thresh[ihru],
                net_snow[ii],
                scrv[ii],
                pksv[ii],
                snowcov_areasv[ii],
            )

            print("   snorun 9: pkwater_equiv = ", pkwater_equiv[ii])

            #       ! HRU STEP 3 - COMPUTE THE NEW ALBEDO
            #       !**********************************************************

            #       ! Compute albedo if there is any snowpack
            #       CALL snalbedo(Newsnow(i), Iso(i), Lst(i), Snsv(i), &
            #  &                  Prmx(i), Pptmix(i), Albset_rnm, Net_snow(i), &
            #  &                  Albset_snm, Albset_rna, Albset_sna, Albedo(i), &
            #  &                  Int_alb(i), Salb(i), Slst(i))
            #     ENDIF

            # ---1---2---3---4---5---6
            int_alb[ii], lst[ii], salb[ii], snsv[ii], albedo[ii] = snalbedo(
                newsnow[ii],
                iso[ihru],
                lst[ii],
                snsv[ii],
                prmx[ii],
                pptmix[ii],
                albset_rnm,
                net_snow[ii],
                albset_snm,
                albset_rna,
                albset_sna,
                int_alb[ii],
                salb[ii],
                slst[ii],
            )

        print(
            "   snorun 10: pkwater_equiv =",
            pkwater_equiv[ii],
            "snowmelt =",
            snowmelt[ii],
            "snow_evap =",
            snow_evap[ii],
        )

        #     ! If there is still a snowpack or glacier
        #     IF ( Pkwater_equiv(i)>0.0D0 .OR. Active_glacier>OFF ) THEN
        # ---1---2---3---4---5---6
        if pkwater_equiv[ii] > 0.0 or active_glacier > 0:
            #       ! HRU STEP 4 - DETERMINE RADIATION FLUXES AND SNOWPACK
            #       !              STATES NECESSARY FOR ENERGY BALANCE
            #       !**********************************************************

            #       ! Set the emissivity of the air to the emissivity when there
            #       ! is no precipitation
            #       emis = Emis_noppt(i) ! [fraction of radiation]
            emis = emis_noppt[ihru]
            #       ! Could use equation from Swinbank 63 using Temp, a is -13.638, b is 6.148
            #       ! temparature is halfway between the minimum and average temperature for the day
            #       !temp = (Tminc(i)+Tavgc(i))*0.5
            #       !emis = ((temp+273.16)**(Emis_coefb-4.0))*(10.0**(Emis_coefa+1.0))/5.670373E~H~R8 ! /by Stefan Boltzmann in SI units
            #       ! If there is any precipitation in the HRU, reset the
            #       ! emissivity to 1
            #       IF ( Hru_ppt(i)>0.0 ) emis = 1.0 ! [fraction of radiation]
            if hru_ppt[ii] > 0.0:
                emis = 1.0
            #       ! Save the current value of emissivity
            #       esv = emis ! [fraction of radiation]
            esv = emis
            #       ! Set the convection-condensation for a half-day interval
            #       cec = Cecn_coef(i, Nowmonth)*0.5 ! [cal/(cm^2 degC)]
            #                                        ! or [Langleys / degC]
            cec = cecn_coef[imon, ihru] * 0.5
            #       ! If the land cover is trees, reduce the convection-
            #       ! condensation parameter by half
            #       IF ( Cov_type(i)>SHRUBS ) cec = cec*0.5 ! [cal/(cm^2 degC)] RSR: cov_type=4 is valid for trees (coniferous)
            #                                               ! or [Langleys / degC]
            if cov_type[ihru] > SHRUBS:
                cec = cec * 0.5
            #       ! Check whether to force spring melt
            #       ! Spring melt is forced if time is before the melt-force
            #       ! day and after the melt-look day (parameters)
            #       ! If between these dates, the spring melt applies if the
            #       ! snowpack temperature is above or equal to 0
            #       ! for more than 4 cycles of the snorun function

            #       ! If before the first melt-force day
            #       IF ( Iso(i)==1 ) THEN
            #         ! If after the first melt-look day
            #         IF ( Mso(i)==2 ) THEN
            # ---1---2---3---4---5---6
            if iso[ii] == 1:
                if mso[ii] == 2:
                    #           ! Melt season is determined by the number of days the
                    #           ! snowpack is above 0 degrees C.  The first time that
                    #           ! the snowpack is isothermal at 0 degrees C for more
                    #           ! than 4 days is the beginning of snowmelt season.
                    #           ! 2 options below (if-then, else)

                    #           ! (1) The snowpack temperature is 0 degrees
                    #           IF ( Pk_temp(i)>=0.0 ) THEN
                    if pk_temp[ii] >= 0.0:
                        # ---1---2---3---4---5---6
                        #             ! Increment the number of days that the snowpack
                        #             ! has been isothermal at 0 degrees C
                        #             Lso(i) = Lso(i) + 1 ! [days]
                        lso[ii] = lso[ii] + 1
                        #             ! If the snowpack temperature has been 0 or greater
                        #             ! for more than 4 cycles
                        #             IF ( Lso(i)>4 ) THEN
                        if lso[ii] > 4:
                            #               ! Set the melt-force flag and reset counter
                            #               Iso(i) = 2 ! [flag]
                            #               Lso(i) = 0 ! [days]
                            iso[ii] = 2
                            lso[ii] = 0
                #             ENDIF

                #           ! (2) The snowpack temperature is less than 0 degrees
                #           ELSE
                # ---1---2---3---4---5---6
                else:
                    #             ! Reset the counter for days snowpack temperature is above 0
                    #             Lso(i) = 0 ! [days]
                    lso[ii] = 0

        #           ENDIF
        #         ENDIF
        #       ENDIF

        #       ! Compute energy balance for night period
        #       ! niteda is a flag indicating nighttime (1) or daytime (2)
        #       ! set the flag indicating night time
        # ---1---2---3---4---5---6
        #       niteda = 1 ! [flag]
        niteda = 1
        #       ! temparature is halfway between the minimum and average temperature
        #       ! for the day
        #       temp = (Tminc(i)+Tavgc(i))*0.5
        tminc = (tminf - 32.0) * 5.0 / 9.0
        tmaxc = (tmaxf - 32.0) * 5.0 / 9.0
        tavgc = (tminc + tmaxc) / 2.0
        temp = (tminc + tavgc) * 0.5

        #       IF ( Pkwater_equiv(i)>0.0D0 ) THEN
        if pkwater_equiv[ii] > 0.0:
            #         ! The incoming shortwave radiation is the HRU radiation
            #         ! adjusted by the albedo (some is reflected back into the
            #         ! atmoshphere) and the transmission coefficient (some is
            #         ! intercepted by the winter vegetative canopy)
            #         swn = Swrad(i)*(1.0-Albedo(i))*Rad_trncf(i) ! [cal/cm^2]
            #                                                     ! or [Langleys]
            swn = swrad[ii] * (1.0 - albedo[ii]) * rad_trncf[ihru]
            #         ! Calculate the new snow depth (Riley et al. 1973)
            #         ! RSR: the following 3 lines of code were developed by Rob Payn, 7/10/2013
            #         ! The snow depth depends on the previous snow pack water
            #         ! equivalent plus the new net snow
            #         Pss(i) = Pss(i) + DBLE( Net_snow(i) ) ! [inches]
            pss[ii] = pss[ii] + net_snow[ii]
            #         dpt_before_settle = Pk_depth(i) + DBLE(Net_snow(i))/DBLE(Den_init(i))
            dpt_before_settle = pk_depth[ii] + net_snow[ii] / den_init[0]
            #         dpt1 = dpt_before_settle + DBLE(Settle_const(i)) * ((Pss(i)/DBLE(Den_max(i))) - dpt_before_settle)
            dpt1 = dpt_before_settle + settle_const[0] * (
                pss[ii] / den_max[ihru] - dpt_before_settle
            )
            # !            dpt1 = Pk_depth(i) + (Net_snow(i)/DBLE(Den_init(i))) + &
            # !                   DBLE(Settle_const(i)) * ((Pss(i)/DBLE(Den_max(i))) - Pk_depth(i))
            # !            dpt1 = ((Net_snow(i)/DBLE(Den_init(i)))+ (Settle_const(i)/Den_max(i)*Pss(i))+Pk_depth(i))*(1.0/(1.0+Settle_const(i))) ! [inches]
            #         ! RAPCOMMENT - CHANGED TO THE APPROPRIATE FINITE DIFFERENCE
            #         !             APPROXIMATION OF SNOW DEPTH
            #         Pk_depth(i) = dpt1 ! [inches]
            pk_depth[ii] = dpt1

            #         ! Calculate the snowpack density
            #         IF ( dpt1>0.0D0 ) THEN
            #           Pk_den(i) = SNGL( Pkwater_equiv(i)/dpt1 )
            #         ELSE
            #           Pk_den(i) = 0.0
            #         ENDIF
            if dpt1 > 0.0:
                pk_den[ii] = pkwater_equiv[ii] / dpt1
            else:
                pk_den[ii] = 0.0
            #                              ! [inch water equiv / inch depth]

            #         ! The effective thermal conductivity is approximated
            #         ! (empirically) as 0.0077 times (snowpack density)^2
            #         ! [cal / (sec g degC)] Therefore, the effective
            #         ! conductivity term (inside the square root) in the
            #         ! equation for conductive heat exchange can be
            #         ! calculated as follows (0.0077*pk_den^2)/(pk_den*0.5)
            #         ! where 0.5 is the specific heat of ice [cal / (g degC)]
            #         ! this simplifies to the following
            #         effk = 0.0154*Pk_den(i) ! [unitless]
            effk = 0.0154 * pk_den[ii]
            #         ! 13751 is the number of seconds in 12 hours over pi
            #         ! So for a half day, to calculate the conductive heat
            #         ! exchange per cm snow per cm^2 area per degree
            #         ! temperature difference is the following
            #         ! In effect, multiplying cst times the temperature
            #         ! gradient gives the heatexchange by heat conducted
            #         ! (calories) per square cm of snowpack
            #         cst = Pk_den(i)*(SQRT(effk*13751.0)) ! [cal/(cm^2 degC)]
            #                                              ! or [Langleys / degC]
            cst = pk_den[ii] * math.sqrt(effk * 13751.0)

            #         ! no shortwave (solar) radiation at night
            #         sw = 0.0 ! [cal / cm^2] or [Langleys]
            sw = 0.0
            #         ! calculate the night time energy balance
            #         CALL snowbal(niteda, Tstorm_mo(i,Nowmonth), Iasw(i), &
            #  &                   temp, esv, Hru_ppt(i), trd, Emis_noppt(i), &
            #  &                   Canopy_covden(i), cec, Pkwater_equiv(i), &
            #  &                   Pk_def(i), Pk_temp(i), Pk_ice(i), Freeh2o(i), &
            #  &                   Snowcov_area(i), Snowmelt(i), Pk_depth(i), &
            #  &                   Pss(i), Pst(i), Pk_den(i), cst, cals, sw, &
            #  &                   Freeh2o_cap(i), Den_max(i), not_a_glacier_hru)
            (
                iasw[ii],
                pkwater_equiv[ii],
                pk_def[ii],
                pk_temp[ii],
                pk_ice[ii],
                freeh2o[ii],
                snowmelt[ii],
                pk_depth[ii],
                pss[ii],
                pst[ii],
                pk_den[ii],
                cals,
            ) = snowbal(
                niteda,
                tstorm_mo[imon, ihru],
                iasw[ii],
                temp,
                esv,
                hru_ppt[ii],
                trd,
                emis_noppt[ihru],
                canopy_covden[ii],
                cec,
                pkwater_equiv[ii],
                pk_def[ii],
                pk_temp[ii],
                pk_ice[ii],
                freeh2o[ii],
                snowcov_area[ii],
                snowmelt[ii],
                pk_depth[ii],
                pss[ii],
                pst[ii],
                pk_den[ii],
                cst,
                sw,
                freeh2o_cap[ihru],
                den_max[0],
                NOT_A_GLACIER_HRU,
            )
            #         ! track total heat flux from both night and day periods
            #         Tcal(i) = cals ! [cal/cm^2] or [Langleys]
            tcal[ii] = cals
        #       ENDIF
        #       iswn  = 0.0
        iswn = 0.0

        #       IF ( Active_glacier>OFF ) THEN
        #         IF ( Glacrcov_area(i)>0.0 ) THEN
        #           iswn = Swrad(i)*(1.0-Glacr_albedo(i))*Rad_trncf(i) ! [cal/cm^2] !want bare ice albedo
        #                                                              ! or [Langleys]
        #           ! Calculate the Glacier icepack density
        #           !
        #           ! The effective thermal conductivity is approximated
        #           ! (empirically) as 0.0077 times (snowpack density)^2 cal/(cm sec degC)
        #           ! from Oke 1987
        #           ! ice is 2.1 W/(m degC) = 0.021 W/(cm deg C) = 0.00502 cal/(cm sec degC)
        #           ! = 0.00597 times (0.917**2),
        #           ! firn (old snow density .5) is closer to 0.0042 W/(cm deg C) = 0.00401 times (0.5**2)
        #           !  Therefore, the effective
        #           ! conductivity term (inside the square root) in the
        #           ! equation for conductive heat exchange can be
        #           ! calculated as follows (0.0597*pk_den^2)/(pk_den*0.5)
        #           ! where 0.5 is the specific heat of ice [cal / (g degC)]
        #           ! this simplifies to the following
        #           ! might want to use 0.005*2 = 0.01 half way between if doing mix of firn and ice
        #           ieffk = 0.01194*Glacr_pk_den(i) ! [unitless]
        #           icst = Glacr_pk_den(i)*(SQRT(ieffk*13751.0)) ! [cal/(cm^2 degC)]
        #                                                        ! or [Langleys / degC]
        #           isw = 0.0 ! [cal / cm^2] or [Langleys]
        #           CALL snowbal(niteda, Tstorm_mo(i,Nowmonth), Iasw(i), &
        #  &                     temp, esv, Hru_ppt(i), trd, Emis_noppt(i), &
        #  &                     Canopy_covden(i), cec, Glacr_pkwater_equiv(i), &
        #  &                     Glacr_pk_def(i), Glacr_pk_temp(i), Glacr_pk_ice(i), Glacr_freeh2o(i), &
        #  &                     Glacrcov_area(i), Glacrmelt(i), Glacr_pk_depth(i), &
        #  &                     Glacr_pss(i), Glacr_pst(i), Glacr_pk_den(i), icst, icals, isw, &
        #  &                     Glacr_freeh2o_capm(i), Den_max(i), i)
        #         ENDIF
        #       ENDIF

        #       ! Compute energy balance for day period
        #       ! set the flag indicating daytime
        #       niteda = 2 ! [flag]
        # ---1---2---3---4---5---6
        niteda = 2
        #       ! temparature is halfway between the maximum and average
        #       ! temperature for the day
        #       temp = (Tmaxc(i)+Tavgc(i))*0.5 ! [degrees C]
        temp = (tmaxc[ii] + tavgc[ii]) * 0.5

        #       IF ( Pkwater_equiv(i)>0.0D0 ) THEN !(if the snowpack still exists)
        if pkwater_equiv[ii] > 0.0:
            #         ! set shortwave radiation as calculated earlier
            #         sw = swn ! [cal/cm^2] or [Langleys]
            sw = swn
            #         CALL snowbal(niteda, Tstorm_mo(i,Nowmonth), Iasw(i), &
            #  &                   temp, esv, Hru_ppt(i), trd, Emis_noppt(i), &
            #  &                   Canopy_covden(i), cec, Pkwater_equiv(i), &
            #  &                   Pk_def(i), Pk_temp(i), Pk_ice(i), Freeh2o(i), &
            #  &                   Snowcov_area(i), Snowmelt(i), Pk_depth(i), &
            #  &                   Pss(i), Pst(i), Pk_den(i), cst, cals, sw, &
            #  &                   Freeh2o_cap(i), Den_max(i), not_a_glacier_hru)
            (
                iasw[ii],
                pkwater_equiv[ii],
                pk_def[ii],
                pk_temp[ii],
                pk_ice[ii],
                freeh2o[ii],
                snowmelt[ii],
                pk_depth[ii],
                pss[ii],
                pst[ii],
                pk_den[ii],
                cals,
            ) = snowbal(
                niteda,
                tstorm_mo[imon, ihru],
                iasw[ii],
                temp,
                esv,
                hru_ppt[ii],
                trd,
                emis_noppt[ihru],
                canopy_covden[ii],
                cec,
                pkwater_equiv[ii],
                pk_def[ii],
                pk_temp[ii],
                pk_ice[ii],
                freeh2o[ii],
                snowcov_area[ii],
                snowmelt[ii],
                pk_depth[ii],
                pss[ii],
                pst[ii],
                pk_den[ii],
                cst,
                sw,
                freeh2o_cap[ihru],
                den_max[0],
                NOT_A_GLACIER_HRU,
            )

            #         ! track total heat flux from both night and day periods
            #         Tcal(i) = Tcal(i) + cals ! [cal/cm^2] or [Langleys]
            tcal[ii] = tcal[ii] + cals
        #       ENDIF
        #       ! Compute energy balance for day period (if glacier exists)
        #       IF ( Active_glacier>OFF ) THEN
        #         IF ( Glacrcov_area(i)>0.0 ) THEN
        #           ! set shortwave radiation as calculated earlier
        #           isw = iswn ! [cal/cm^2] or [Langleys]
        #           CALL snowbal(niteda, Tstorm_mo(i,Nowmonth), Iasw(i), &
        #  &                     temp, esv, Hru_ppt(i), trd, Emis_noppt(i), &
        #  &                     Canopy_covden(i), cec, Glacr_pkwater_equiv(i), &
        #  &                     Glacr_pk_def(i), Glacr_pk_temp(i), Glacr_pk_ice(i), Glacr_freeh2o(i), &
        #  &                     Glacrcov_area(i), Glacrmelt(i), Glacr_pk_depth(i), &
        #  &                     Glacr_pss(i), Glacr_pst(i), Glacr_pk_den(i), icst, icals, isw, &
        #  &                     Glacr_freeh2o_capm(i), Den_max(i), i)
        #         ENDIF
        #       ENDIF

        #       !  HRU STEP 5 - CALCULATE SNOWPACK LOSS TO EVAPORATION
        #       !********************************************************

        #       ! Compute snow evaporation (if there is still a snowpack)
        #       ! Some of the calculated evaporation can come from interception
        #       ! rather than the snowpack.  Therefore, the effects of
        #       ! interception must be evaluated.
        #       IF ( Pkwater_equiv(i)>0.0D0 ) THEN
        # ---1---2---3---4---5---6
        if pkwater_equiv[ii] > 0.0:
            #         ! Snow can evaporate when transpiration is not occuring
            #         ! or when transpiration is occuring with cover types of
            #         ! bare soil or grass
            #         IF ( Transp_on(i)==0 .OR. (Transp_on(i)==1 .AND. Cov_type(i)<SHRUBS) ) &
            if transp_on[ii] == 0 or (transp_on[ii] == 1 and cov_type[ihru] < SHRUBS):
                #  &           CALL snowevap(Potet_sublim(i), Potet(i), Snowcov_area(i), &
                #  &                         Snow_evap(i), Pkwater_equiv(i), Pk_ice(i), &
                #  &                         Pk_def(i), Freeh2o(i), Pk_temp(i), Hru_intcpevap(i))
                snowevap(
                    potet_sublim[ii],
                    potet[ii],
                    sowcov_area[ii],
                    snow_evap[ii],
                    pkwater_equiv[ii],
                    pk_ice[ii],
                    pk_def[ii],
                    freeh2o[ii],
                    pk_temp[ii],
                    hru_intcpevap[ii],
                )
        #       ELSEIF ( Pkwater_equiv(i)<0.0D0 ) THEN
        elif pkwater_equiv[ii] < 0.0:
            #         IF ( Print_debug>DEBUG_less ) THEN
            #           IF ( Pkwater_equiv(i)<-DNEARZERO ) PRINT *, 'snowpack issue 3, negative pkwater_equiv, &
            #  &             HRU:', i, ' value:', Pkwater_equiv(i)
            #         ENDIF
            #         Pkwater_equiv(i) = 0.0D0 ! just to be sure negative values are ignored
            pkwater_equiv[ii] = 0.0
        #       ENDIF
        #       IF ( Active_glacier>OFF ) THEN
        #         IF ( Glacrcov_area(i)>0.0 ) &
        #  &           CALL snowevap(Potet_sublim(i), Potet(i), Glacrcov_area(i), &
        #  &                         Glacr_evap(i), Glacr_pkwater_equiv(i), Glacr_pk_ice(i), &
        #  &                         Glacr_pk_def(i), Glacr_freeh2o(i), Glacr_pk_temp(i), Hru_intcpevap(i))
        #       ENDIF

        # ! LAST check to clear out all arrays if packwater is gone
        #     IF ( Pkwater_equiv(i)<=0.0D0 ) THEN
        if pkwater_equiv[ii] <= 0.0:
            #       IF ( Print_debug>DEBUG_less ) THEN
            #         IF ( Pkwater_equiv(i)<-DNEARZERO ) &
            #  &           PRINT *, 'Snowpack problem, pkwater_equiv negative, HRU:', i, ' value:', Pkwater_equiv(i)
            #       ENDIF
            #       Pkwater_equiv(i) = 0.0D0 ! just to be sure negative values are ignored
            pkwater_equiv[ii] = 0.0
            #       ! Snowpack has been completely depleted, reset all states
            #       ! to no-snowpack values
            #       Pk_depth(i) = 0.0D0
            #       Pss(i) = 0.0D0
            #       Snsv(i) = 0.0
            #       Lst(i) = 0
            #       Pst(i) = 0.0D0
            #       Iasw(i) = 0
            #       Albedo(i) = 0.0
            #       Pk_den(i) = 0.0
            #       Snowcov_area(i) = 0.0
            #       Pk_def(i) = 0.0
            #       Pk_temp(i) = 0.0
            #       Pk_ice(i) = 0.0
            #       Freeh2o(i) = 0.0
            #       Snowcov_areasv(i) = 0.0 ! rsr, not in original code
            #       Ai(i) = 0.0D0
            #       Frac_swe(i) = 0.0
            #       Scrv(i) = 0.0D0
            #       Pksv(i) = 0.0D0
            pk_depth[ii] = 0.0
            pss[ii] = 0.0
            snsv[ii] = 0.0
            lst[ii] = 0
            pst[ii] = 0.0
            iasw[ii] = 0
            albedo[ii] = 0.0
            pk_den[ii] = 0.0
            snowcov_area[ii] = 0.0
            pk_def[ii] = 0.0
            pk_temp[ii] = 0.0
            pk_ice[ii] = 0.0
            freeh2o[ii] = 0.0
            snowcov_areasv[ii] = 0.0
            ai[ii] = 0.0
            frac_swe[ii] = 0.0
            scrv[ii] = 0.0
            pksv[ii] = 0.0
        #     ENDIF
        #     frac = 1.0
        frac = 1.0
        #     IF ( Active_glacier>OFF ) THEN
        #       IF ( Glacr_pkwater_equiv(i)>0.0D0 ) THEN
        #         Glacr_pk_depth(i) = Glacr_pkwater_equiv(i)/DBLE(Glacr_pk_den(i))
        #       ELSE
        #         CALL glacr_states_to_zero(i,0)
        #       ENDIF
        #       Basin_glacrb_melt = Basin_glacrb_melt + DBLE( Glacrb_melt(i)*Hru_area(i) )
        #       Basin_glacrevap = Basin_glacrevap + DBLE( Glacr_evap(i)*Hru_area(i) )
        #       IF ( Active_glacier==1 ) frac = (1.0 - Glacier_frac(i))
        #       IF ( Active_glacier==2 ) frac = (1.0 - Glrette_frac(i))
        #     ENDIF

        ii += 1

    # !****************************************************************
    # !   Declared Variables
    # INTEGER :: Yrdays5
    # INTEGER, SAVE, ALLOCATABLE :: Pptmix_nopack(:), Lst(:)
    # INTEGER, SAVE, ALLOCATABLE :: Iasw(:), Iso(:), Mso(:), Lso(:)
    # DOUBLE PRECISION, SAVE :: Basin_snowmelt, Basin_pweqv, Basin_tcal
    # DOUBLE PRECISION, SAVE :: Basin_snowcov, Basin_snowevap
    # DOUBLE PRECISION, SAVE :: Basin_snowdepth, Basin_pk_precip
    # REAL, SAVE, ALLOCATABLE :: Snowmelt(:), Snow_evap(:)
    # REAL, SAVE, ALLOCATABLE :: Albedo(:), Pk_temp(:), Pk_den(:)
    # REAL, SAVE, ALLOCATABLE :: Pk_def(:), Pk_ice(:), Freeh2o(:)
    # REAL, SAVE, ALLOCATABLE :: Snowcov_area(:), Tcal(:)
    # REAL, SAVE, ALLOCATABLE :: Snsv(:), Pk_precip(:)
    # REAL, SAVE, ALLOCATABLE :: Frac_swe(:)
    # DOUBLE PRECISION, SAVE, ALLOCATABLE :: Pk_depth(:), Pkwater_ante(:), Ai(:)
    # DOUBLE PRECISION, SAVE :: Basin_glacrevap, Basin_snowicecov, Basin_glacrb_melt
    # REAL, SAVE, ALLOCATABLE :: Glacrmelt(:), Glacr_evap(:), Glacr_albedo(:), Glacr_pk_den(:)
    # REAL, SAVE, ALLOCATABLE :: Glacr_pk_ice(:), Glacr_freeh2o(:), Glacrcov_area(:)
    # REAL, SAVE, ALLOCATABLE :: Glacrb_melt(:), Glacr_pk_def(:), Glacr_pk_temp(:), Ann_tempc(:)
    # REAL, SAVE, ALLOCATABLE :: Glacr_air_5avtemp1(:), Glacr_air_deltemp(:), Glacr_air_5avtemp(:)
    # REAL, SAVE, ALLOCATABLE :: Glacr_5avsnow1(:), Glacr_5avsnow(:), Glacr_delsnow(:), Glacr_freeh2o_capm(:)
    # DOUBLE PRECISION, SAVE, ALLOCATABLE :: Glacr_pkwater_ante(:), Glacr_pkwater_equiv(:)
    # DOUBLE PRECISION, SAVE, ALLOCATABLE :: Glacr_pk_depth(:), Glacr_pss(:), Glacr_pst(:)

    return (
        pptmix_nopack,
        lst,
        iasw,
        iso,
        mso,
        lso,
        snowmelt,
        snow_evap,
        albedo,
        pk_temp,
        pk_den,
        pk_def,
        pk_ice,
        freeh20,
        snowcov_area,
        tcal,
        snsv,
        pk_precip,
        frac_swe,
        pk_depth,
        ai,
        pkwater_equiv,
        scrv,
        pksv,
        snowcov_areasv,
        frac_swe,
    )

In [ ]:
dates = synced_vals["date"].values
basin_horad = synced_vals["basin_horad"].values
orad = synced_vals["orad"].values
newsnow = synced_vals["newsnow"].values
net_snow = synced_vals["net_snow"].values
net_rain = synced_vals["net_rain"].values
pptmix = synced_vals["pptmix"].values
tmaxf = synced_vals["tmaxf"].values
tminf = synced_vals["tminf"].values
prmx = synced_vals["prmx"].values
net_ppt = synced_vals["net_ppt"].values
hru_ppt = synced_vals["hru_ppt"].values
swrad = synced_vals["swrad"].values
canopy_covden = synced_vals["canopy_covden"].values
transp_on = synced_vals["transp_on"].values

params = read_param_file(pfn)
v = params[1]
hru_area = v["hru_area"]
hru_lat = v["hru_lat"]
melt_force = v["melt_force"]
melt_look = v["melt_look"]
tmax_allsnow = v["tmax_allsnow"]
freeh2o_cap = v["freeh2o_cap"]
den_max = v["den_max"]
snarea_curve = v["snarea_curve"]
hru_deplcrv = v["hru_deplcrv"]
snarea_thresh = v["snarea_thresh"]
albset_rnm = v["albset_rnm"]
albset_sna = v["albset_sna"]
albset_snm = v["albset_snm"]
albset_rna = v["albset_rna"]
cecn_coef = v["cecn_coef"]
emis_noppt = v["emis_noppt"]
cov_type = v["cov_type"]
rad_trncf = v["rad_trncf"]
den_init = v["den_init"]
settle_const = v["settle_const"]
tstorm_mo = v["tstorm_mo"]

(
    pptmix_nopack,
    lst,
    iasw,
    iso,
    mso,
    lso,
    snowmelt,
    snow_evap,
    albedo,
    pk_temp,
    pk_den,
    pk_def,
    pk_ice,
    freeh20,
    snowcov_area,
    tcal,
    snsv,
    pk_precip,
    frac_swe,
    pk_depth,
    ai,
    pkwater_equiv,
    scrv,
    pksv,
    snowcov_areasv,
    frac_swe,
) = snorun(
    dates,
    basin_horad,
    orad,
    newsnow,
    tmaxf,
    tminf,
    net_snow,
    net_rain,
    pptmix,
    prmx,
    net_ppt,
    hru_ppt,
    swrad,
    canopy_covden,
    transp_on,
    hru_area,
    hru_lat,
    melt_force,
    melt_look,
    tmax_allsnow,
    freeh2o_cap,
    den_max,
    hru_deplcrv,
    snarea_thresh,
    albset_rnm,
    albset_sna,
    albset_snm,
    albset_rna,
    cecn_coef,
    emis_noppt,
    cov_type,
    rad_trncf,
    den_init,
    settle_const,
    tstorm_mo,
)

In [ ]:
dates = synced_vals["date"].values

In [ ]:
err = synced_vals["pkwater_equiv"] - pkwater_equiv
print(sum(synced_vals["pkwater_equiv"].values), sum(pkwater_equiv))

In [ ]:
# ii = 0
# for date in dates:
# #     jday = day_of_year(date)
#     imon = date.month - 1
#     if err[ii] < -0.001:
#         print(date, err[ii])
#     ii += 1

In [ ]:
plt.plot(dates, err)
plt.xticks(rotation=30, ha="right")
plt.xlabel("Date")
plt.ylabel("Error pkwater_equiv (Inches per unit area)")

In [ ]:
plt.scatter(synced_vals["pkwater_equiv"].values, pkwater_equiv)
plt.xlabel("PRMS computed pkwater_equiv (Inches per unit area)")
plt.ylabel("Python computed pkwater_equiv (Inches per unit area)")

In [ ]:
err = synced_vals["pptmix_nopack"] - pptmix_nopack
print(sum(synced_vals["pptmix_nopack"].values), sum(pptmix_nopack))

In [ ]:
plt.plot(dates, err)
plt.xticks(rotation=30, ha="right")
plt.xlabel("Date")
plt.ylabel("Error pptmix_nopack (Unitless)")

In [ ]:
plt.scatter(synced_vals["pptmix_nopack"].values, pptmix_nopack)
plt.xlabel("PRMS computed pptmix_nopack (Unitless)")
plt.ylabel("Python computed pptmix_nopack (Unitless)")

In [ ]:
err = synced_vals["lst"] - lst
print(sum(synced_vals["lst"].values), sum(lst))

In [ ]:
plt.plot(dates, err)
plt.xticks(rotation=30, ha="right")
plt.xlabel("Date")
plt.ylabel("Error lst ()")

In [ ]:
plt.scatter(synced_vals["lst"].values, lst)
plt.xlabel("PRMS computed lst ()")
plt.ylabel("Python computed lst ()")